# Computação Gráfica e Realidade Aumentada 
# Visualização Computacional 
## 2022/2023
<author>Authors:</author>
<author>André Araújo 2018276072 </author>
<author>Bernardo Leite 2017268958 </author>
<author>Ângelo Huang 2018292729 </author>
<h1>Assignment 3</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgratexture.js"></script>

### Shaders

Shader básico sem utilização de texturas nem iluminação.

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main() {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

Shader para utilização apenas de texturas.

In [ ]:
%%html
<script id="my-vertex-shaderT" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute vec2 in_texcoords;
uniform mat4 MVP;

varying  vec3 ex_Color;
varying highp vec2 vTextureCoord;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    vTextureCoord = in_texcoords;
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shaderT" type="x-shader/x-fragment">
precision mediump float;
varying highp vec2 vTextureCoord;

uniform sampler2D uSampler;
varying  vec3 ex_Color;

void main() {
      gl_FragColor = texture2D(uSampler, vTextureCoord);
       //gl_FragColor = vec4(ex_Color,1.0);
}
</script>

Shader para utilização apenas de luz direcional.

In [ ]:
%%html
<script id="my-vertex-shader-light" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_normals;
uniform mat4 MVP;
uniform mat3 NormalMatrix; // to transform normals, pre-perspective

varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec3 ex_Position;

void main() {
  
    Normal = normalize(NormalMatrix*in_normals);
    
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    
    ex_Position = vec3(gl_Position);
    
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-light" type="x-shader/x-fragment">
precision mediump float;

uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightPosition;
uniform vec3 ViewPosition;
//uniform vec3 HalfVector; // surface orientation for shiniest spots
uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

varying  vec3 Normal; // Surface normal, interpolated between vertices
varying  vec3 ex_Color;
varying  vec3 ex_Position;

void main() {
  
    vec3 lightDir = LightPosition - ex_Position;
    vec3 viewDir = ViewPosition - ex_Position;// Para calcular half vector
    
    vec3 HVDir = normalize(lightDir + viewDir);// Half vector direction
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    
    float diffuse = max(0.0, dot(Normal, LightPosition));
    float specular = max(0.0, dot(Normal, HVDir));
    
    // surfaces facing away from the light
    // won`t be lit by the directional light
    if(diffuse == 0.0){
        specular = 0.0;
    }else{
        // sharpen the highlight
        specular = pow(specular, Shininess);
    }
    
    vec3 scatteredLight = Ambient + LightColor*diffuse;
    vec3 reflectedLight = LightColor*specular*Strength;
    
    // don`t modulate the underlying color with reflected light,
    // only with scattered light
    
    vec3 rgb = min(ex_Color.rgb*scatteredLight+reflectedLight, vec3(1.0));
    
    gl_FragColor = vec4(rgb,1.0);
    
}
</script>

Shader para utilização de texturas e luz direcional com componente especular.

In [ ]:
%%html
<script id="my-vertex-shader-D" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_normals;
attribute vec2 in_texcoords;
uniform mat4 MVP;
uniform mat3 NormalMatrix; // to transform normals, pre-perspective

varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
    Normal = normalize(NormalMatrix*in_normals);
    vTextureCoord = in_texcoords;
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    ex_Position = vec3(gl_Position);
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-D" type="x-shader/x-fragment">
precision mediump float;

uniform sampler2D uSampler;
uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightPosition;
uniform vec3 ViewPosition;
//uniform vec3 HalfVector; // surface orientation for shiniest spots
uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

varying  vec3 Normal; // Surface normal, interpolated between vertices
varying  vec3 ex_Color;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
  
    vec3 lightDir = LightPosition - ex_Position;
    vec3 viewDir = ViewPosition - ex_Position;// Para calcular half vector
    
    vec3 HVDir = normalize(lightDir + viewDir);// Half vector direction
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    
    float diffuse = max(0.0, dot(Normal, LightPosition));
    float specular = max(0.0, dot(Normal, HVDir));
    
    // surfaces facing away from the light
    // won`t be lit by the directional light
    if(diffuse == 0.0){
        specular = 0.0;
    }else{
        // sharpen the highlight
        specular = pow(specular, Shininess);
    }
    
    vec3 scatteredLight = Ambient + LightColor*diffuse;
    vec3 reflectedLight = LightColor*specular*Strength;
    
    // don`t modulate the underlying color with reflected light,
    // only with scattered light
    
    vec4 corTex = texture2D(uSampler, vTextureCoord);
    vec3 rgb = min(corTex.rgb*scatteredLight+reflectedLight, vec3(1.0));
    gl_FragColor = vec4(rgb,1.0);
    //gl_FragColor = vec4(LightPosition, 1.0);
    
}
</script>

Shader para utilização de texturas e luz direcional sem componente especular.

In [ ]:
%%html
<script id="my-vertex-shader-NoSpecular" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_normals;
attribute vec2 in_texcoords;
uniform mat4 MVP;
uniform mat3 NormalMatrix; // to transform normals, pre-perspective

varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
    Normal = normalize(NormalMatrix*in_normals);
    vTextureCoord = in_texcoords;
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    ex_Position = vec3(gl_Position);
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-NoSpecular" type="x-shader/x-fragment">
precision mediump float;

uniform sampler2D uSampler;
uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightPosition;
uniform vec3 ViewPosition;
//uniform vec3 HalfVector; // surface orientation for shiniest spots
uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

varying  vec3 Normal; // Surface normal, interpolated between vertices
varying  vec3 ex_Color;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
  
    vec3 lightDir = LightPosition - ex_Position;
    vec3 viewDir = ViewPosition - ex_Position;// Para calcular half vector
    
    vec3 HVDir = normalize(lightDir + viewDir);// Half vector direction
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    
    float diffuse = max(0.0, dot(Normal, LightPosition));
    
    vec3 scatteredLight = Ambient + LightColor*diffuse;
    vec3 reflectedLight = LightColor*0.1;
    
    // don`t modulate the underlying color with reflected light,
    // only with scattered light
    
    vec4 corTex = texture2D(uSampler, vTextureCoord);
    vec3 rgb = min(corTex.rgb*scatteredLight+reflectedLight, vec3(1.0));
    gl_FragColor = vec4(rgb,1.0);
    //gl_FragColor = vec4(LightPosition, 1.0);
    
}
</script>

Shader para utilização de texturas e luz pontual com componente especular.

In [ ]:
%%html
<script id="my-vertex-shader-P" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_normals;
attribute vec2 in_texcoords;
uniform mat4 MVP;
uniform mat3 NormalMatrix; // to transform normals, pre-perspective

varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
    Normal = normalize(NormalMatrix*in_normals);
    vTextureCoord = in_texcoords;
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    ex_Position = vec3(gl_Position);
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-P" type="x-shader/x-fragment">
precision mediump float;

uniform sampler2D uSampler;
uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightPosition;
uniform vec3 ViewPosition;
uniform float LightDistance;
//uniform vec3 HalfVector; // surface orientation for shiniest spots
uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

varying  vec3 Normal; // Surface normal, interpolated between vertices
varying  vec3 ex_Color;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

uniform float ConstantAttenuation;
uniform float LinearAttenuation;
uniform float QuadraticAttenuation;

void main() {
  
    vec3 lightDir = LightPosition - ex_Position;
    vec3 viewDir = ViewPosition - ex_Position;// Para calcular half vector
    
    vec3 HVDir = normalize(lightDir + viewDir);// Half vector direction
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    
    float diffuse = max(0.0, dot(Normal, LightPosition));
    float specular = max(0.0, dot(Normal, HVDir));
    
    // surfaces facing away from the light
    // won`t be lit by the directional light
    if(diffuse == 0.0){
        specular = 0.0;
    }else{
        // sharpen the highlight
        specular = pow(specular, Shininess);
    }
    
    float attenuation = 1.0 /(ConstantAttenuation + LinearAttenuation * LightDistance + QuadraticAttenuation * LightDistance * LightDistance);
    
    vec3 scatteredLight = Ambient + LightColor*diffuse*attenuation;
    vec3 reflectedLight = LightColor*specular*Strength*attenuation;
    
    // don`t modulate the underlying color with reflected light,
    // only with scattered light
    
    vec4 corTex = texture2D(uSampler, vTextureCoord);
    vec3 rgb = min(corTex.rgb*scatteredLight+reflectedLight, vec3(1.0));
    gl_FragColor = vec4(rgb,1.0);
    
}
</script>

Shader para utilização de texturas e luz pontual sem componente especular.

In [ ]:
%%html
<script id="my-vertex-shader-NoSpecularP" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute  vec3 in_normals;
attribute vec2 in_texcoords;
uniform mat4 MVP;
uniform mat3 NormalMatrix; // to transform normals, pre-perspective

varying  vec3 ex_Color;
varying  vec3 Normal;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

void main() {
    Normal = normalize(NormalMatrix*in_normals);
    vTextureCoord = in_texcoords;
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    ex_Position = vec3(gl_Position);
    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader-NoSpecularP" type="x-shader/x-fragment">
precision mediump float;

uniform sampler2D uSampler;
uniform vec3 Ambient;
uniform vec3 LightColor;
uniform vec3 LightPosition;
uniform vec3 ViewPosition;
uniform float LightDistance; // direction toward the light
//uniform vec3 HalfVector; // surface orientation for shiniest spots
uniform float Shininess; // exponent for sharping highlights
uniform float Strength; // extra factor to adjust shininess

varying  vec3 Normal; // Surface normal, interpolated between vertices
varying  vec3 ex_Color;
varying  vec2 vTextureCoord;
varying  vec3 ex_Position;

uniform float ConstantAttenuation;
uniform float LinearAttenuation;
uniform float QuadraticAttenuation;

void main() {
  
    vec3 lightDir = LightPosition - ex_Position;
    vec3 viewDir = ViewPosition - ex_Position;// Para calcular half vector
    
    vec3 HVDir = normalize(lightDir + viewDir);// Half vector direction
    
    // compute cosine of the directions, using dot products,
    // to see how much light would be reflected
    
    float diffuse = max(0.0, dot(Normal, LightPosition));
    
    float attenuation = 1.0 /(ConstantAttenuation + LinearAttenuation * LightDistance + QuadraticAttenuation * LightDistance * LightDistance);
    
    vec3 scatteredLight = Ambient + LightColor*diffuse*attenuation;
    vec3 reflectedLight = LightColor*0.1*attenuation;
    
    // don`t modulate the underlying color with reflected light,
    // only with scattered light
    
    vec4 corTex = texture2D(uSampler, vTextureCoord);
    vec3 rgb = min(corTex.rgb*scatteredLight+reflectedLight, vec3(1.0));
    gl_FragColor = vec4(rgb,1.0);
    
}
</script>

In [ ]:
%%html
<canvas id="myCanvas" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
<script id="myapp">

class cube extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        this.normals = [];
        
        this.Index=0;
        this.positions = [
            glm.vec3( -0.5, -0.5,  0.5 ),
            glm.vec3( -0.5,  0.5,  0.5 ),
            glm.vec3(  0.5,  0.5,  0.5 ),
            glm.vec3(  0.5, -0.5,  0.5 ),
            glm.vec3( -0.5, -0.5, -0.5 ),
            glm.vec3( -0.5,  0.5, -0.5 ),
            glm.vec3(  0.5,  0.5, -0.5 ),
            glm.vec3(  0.5, -0.5, -0.5 )
        ];
        
        this.colors = [
            glm.vec3( 0.0, 0.0, 0.0 ),  // black
            glm.vec3( 1.0, 0.0, 0.0 ),  // red
            glm.vec3( 1.0, 1.0, 0.0 ),  // yellow
            glm.vec3( 0.0, 1.0, 0.0 ),  // green
            glm.vec3( 0.0, 0.0, 1.0 ),  // blue
            glm.vec3( 1.0, 0.0, 1.0 ),  // magenta
            glm.vec3( 1.0, 1.0, 1.0 ),  // white
            glm.vec3( 0.0, 1.0, 1.0 )   // cyan
        ];
        
        this.tcoords = [
            glm.vec2(0.0,0.0),
            glm.vec2(0.0,1.0),
            glm.vec2(1.0,1.0),
            glm.vec2(1.0,0.0)
        ];
        
        this.quad( 0, 1, 2, 3, 0.33, 1.0, 0.66, 1.0, 0.66, 0.75, 0.33, 0.75 );
        this.quad( 3, 2, 6, 7, 0.33, 0.75, 0.66, 0.75, 0.66, 0.50, 0.33, 0.50);
        this.quad( 2, 1, 5, 6, 0.66, 0.75, 0.66, 1.0, 1.0, 1.0, 1.0, 0.75 );
        this.quad( 7, 4, 0, 3, 0.0, 0.75, 0.0, 1.0, 0.33, 1.0, 0.33, 0.75 );
        this.quad( 5, 4, 7, 6, 0.66, 0.25, 0.33, 0.25, 0.33, 0.50, 0.66, 0.50 );
        this.quad( 4, 5, 1, 0, 0.33, 0.25, 0.66, 0.25, 0.66, 0.0, 0.33, 0.0 );
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.numvertices = 36;
    }
    
    quad(a, b, c, d, ax, ay, bx, by, cx, cy, dx, dy)
    {
        //console.log("quad",a,b,c,d);
        
        // Normais do 1º triângulo
        var Ux = this.positions[a].x - this.positions[b].x;
        var Uy = this.positions[a].y - this.positions[b].y;
        var Uz = this.positions[a].z - this.positions[b].z;
        
        var Vx = this.positions[c].x - this.positions[b].x;
        var Vy = this.positions[c].y - this.positions[b].y;
        var Vz = this.positions[c].z - this.positions[b].z;
        
        var U = glm.vec3(Ux,Uy,Uz);
        var V = glm.vec3(Vx,Vy,Vz);
        
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, ax, ay);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);
        
        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, bx, by);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);
           
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords, cx, cy);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);
      
        
        
        // Normais do 2º triângulo
        var Ux = this.positions[c].x - this.positions[d].x;
        var Uy = this.positions[c].y - this.positions[d].y;
        var Uz = this.positions[c].z - this.positions[d].z;    
            
        var Vx = this.positions[a].x - this.positions[d].x;
        var Vy = this.positions[a].y - this.positions[d].y;
        var Vz = this.positions[a].z - this.positions[d].z;
            
        U = glm.vec3(Ux,Uy,Uz);
        V = glm.vec3(Vx,Vy,Vz);
        
        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, ax, ay);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);
       
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords, cx, cy);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);
       
        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, dx, dy);
        this.normals = this.normals.concat(glm.normalize(glm.cross(U,V)).array);

    }
}

class cubeT extends cube{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    } 
}

class cubeText extends cube{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    } 
}

class myapp extends DEECapp{
    counter = 0;
    showt = false;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shaderT").text;
        var vertsrc = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("gallery_1.jpg");
        
        this.cube = new cubeText(this.gl);
        this.cube.setShader(this.shaderprog);
        this.cube.settexture(this.texture);
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(glm.vec3(0,0,0),glm.vec3(0,0,1),glm.vec3(0,1,0))
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.counter++;
        
        var scale = glm.mat4(glm.mat3(2));
        //translação z=-3;
        var transformation = glm.translate(scale,glm.vec3(0,0,3));
        //rotação no eixo yy
        var rot1 = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(0.0,1.0,0.0)));
        //rotação inverso nos eixo xx e zz
        var rot2 = glm.toMat4(glm.angleAxis(glm.radians(45),glm.vec3(1.0,1.0,0.0)));
        transformation = transformation['*'](rot2)['*'](rot1);
        this.cube.setModelTransformation(transformation);
        this.cube.drawit(this.viewM,this.projectionM);
    }
    
}

var app = new myapp('myCanvas');

app.run();

</script>

In [ ]:
%%html
<script id="myapp">
class cube3 extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        
        this.Index=0;
        this.positions = [
            glm.vec3( -5, -5,  5 ),
            glm.vec3( -5,  5,  5 ),
            glm.vec3(  5,  5,  5 ),
            glm.vec3(  5, -5,  5 ),
            glm.vec3( -5, -5, -5 ),
            glm.vec3( -5,  5, -5 ),
            glm.vec3(  5,  5, -5 ),
            glm.vec3(  5, -5, -5 )
        ];
        
        this.colors = [
            glm.vec3( 0.0, 0.0, 0.0 ),  // black
            glm.vec3( 1.0, 0.0, 0.0 ),  // red
            glm.vec3( 1.0, 1.0, 0.0 ),  // yellow
            glm.vec3( 0.0, 1.0, 0.0 ),  // green
            glm.vec3( 0.0, 0.0, 1.0 ),  // blue
            glm.vec3( 1.0, 0.0, 1.0 ),  // magenta
            glm.vec3( 1.0, 1.0, 1.0 ),  // white
            glm.vec3( 0.0, 1.0, 1.0 )   // cyan
        ];
        
        this.tcoords = [
            glm.vec2(0.0,0.0),
            glm.vec2(0.0,1.0),
            glm.vec2(1.0,1.0),
            glm.vec2(1.0,0.0)
        ];
        
      
        this.quad( 1, 0, 3, 2 );
        this.quad( 2, 3, 7, 6 );
        this.quad( 3, 0, 4, 7 );
        this.quad( 6, 5, 1, 2 );
        this.quad( 4, 5, 6, 7 );
        this.quad( 5, 4, 0, 1 );
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d )
    {
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[0].array);
        
        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[1].array);
                        
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
      
        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[0].array);
       
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[2].array);
       
        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[3].array);

    }
    
}

class cubeT3 extends cube3{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

### Cone

In [ ]:
%%html
    
<script id="myapp">

class cone extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        
        var vertices = [];
        var colors = [];
        var disco = [];
        this.texCoords = [];
        this.normals = [];
        
        
        // VERTICES
        // Criar 10 vértices como uma aproximação circular
        for (var j = 0.0; j <= 2*Math.PI; j += 2.0*Math.PI/10){
                disco = disco.concat(1.0*Math.cos(j), 1.0*Math.sin(j), -1.0);
        }
        //console.log(disco);
                 
        
        // Unir pontos do disco ao centro do cone
        for (var j=0; j<31; j=j+3){
            vertices = vertices.concat(disco[j],disco[j+1],disco[j+2]);  
            vertices = vertices.concat(0,0,-1); 
            vertices = vertices.concat(disco[j+3],disco[j+4],disco[j+5]);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([0,0,-1]);
            this.normals = this.normals.concat([0,0,-1]);
            this.normals = this.normals.concat([0,0,-1]);

        }
        
        // Unir pontos do disco ao vertice superior do cone
        for (var j=0; j<31; j=j+3){
            vertices = vertices.concat(disco[j],disco[j+1],disco[j+2]);  
            vertices = vertices.concat(0,0,1); 
            vertices = vertices.concat(disco[j+3],disco[j+4],disco[j+5]);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([0,0,1]);
            this.normals = this.normals.concat([0,0,1]);
            this.normals = this.normals.concat([0,0,1]);

        }
        //console.log(vertices);
        //console.log(colors);
        
        
        // TEXTURE COORDS
        //this.tcoords = [
            //glm.vec2(0.0,0.0),
            //glm.vec2(0.0,1.0),
            //glm.vec2(1.0,0.0)
        //];
        this.tcoords = [
            glm.vec2(0.0,1.0),
            glm.vec2(0.0,0.0),
            glm.vec2(1.0,0.0)
        ];
        
        for (var j=0; j<vertices.length/3; j=j+3){
           this.texCoords = this.texCoords.concat(this.tcoords[0].array); 
           this.texCoords = this.texCoords.concat(this.tcoords[1].array);
           this.texCoords = this.texCoords.concat(this.tcoords[2].array);
        }
        
        this.numvertices = vertices.length;
        
        // VERTICES
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        // COLORS
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // TEXTURE COORDS
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW);
        
        // NORMALS
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        var a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);   
        
    }
}



class coneT extends cone{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    }     
}

class coneText extends cone{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    }  
}

</script>

### Cylinder

In [ ]:
%%html
    
<script id="myapp">

class cilindro extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        
        var vertices = [];
        var colors = [];
        var disco1 = [];
        var disco2 = [];
        this.texCoords = [];
        this.normals = [];
        
        // VERTICES
        // Disco 1 -> Base de cima do cilindo (z=1)
        for (var j = 0.0; j <= 2*Math.PI; j += 2.0*Math.PI/10){
                disco1 = disco1.concat(1.0*Math.cos(j), 1.0*Math.sin(j), 1.0);
        }
        //console.log(disco1);
        
        // Disco 2 -> Base de baixo do cilindo (z=-1)
        for (var j = 0.0; j <= 2*Math.PI; j += 2.0*Math.PI/10){
                disco2 = disco2.concat(1.0*Math.cos(j), 1.0*Math.sin(j), -1.0);
        }
        //console.log(disco2);
        
                
        // Unir pontos do disco1 ao centro da base superior (z=1)
        for (var j=0; j<31; j=j+3){
            vertices = vertices.concat(disco1[j],disco1[j+1],disco1[j+2]);  
            vertices = vertices.concat(0,0,1); 
            vertices = vertices.concat(disco1[j+3],disco1[j+4],disco1[j+5]);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([0,0,1]);
            this.normals = this.normals.concat([0,0,1]);
            this.normals = this.normals.concat([0,0,1]);

        }
        
        // Unir pontos do disco2 ao centro da base inferior (z=-1)
        for (var j=0; j<31; j=j+3){
            vertices = vertices.concat(disco2[j],disco2[j+1],disco2[j+2]);  
            vertices = vertices.concat(0,0,-1); 
            vertices = vertices.concat(disco2[j+3],disco2[j+4],disco2[j+5]);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([0,0,-1]);
            this.normals = this.normals.concat([0,0,-1]);
            this.normals = this.normals.concat([0,0,-1]);
        }
        
        // Unir os pontos entre as duas bases (discos) para criar o cilindro
        for (var j=0; j<31; j=j+3){

            vertices = vertices.concat(disco1[j],disco1[j+1],disco1[j+2]); // primeiro ponto da base de cima 
            vertices = vertices.concat(disco1[j+3],disco1[j+4],disco1[j+5]); // segundo ponto da base de cima 
            vertices = vertices.concat(disco2[j],disco2[j+1],disco2[j+2]);  // primeiro ponto da base de baixo
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([disco1[j],disco1[j+1],0]);
            this.normals = this.normals.concat([disco1[j+3],disco1[j+4],0]);
            this.normals = this.normals.concat([disco2[j],disco2[j+1],0]);
            
            vertices = vertices.concat(disco2[j],disco2[j+1],disco2[j+2]); // primeiro ponto da base de baixo 
            vertices = vertices.concat(disco2[j+3],disco2[j+4],disco2[j+5]); // primeiro ponto da base de baixo 
            vertices = vertices.concat(disco1[j+3],disco1[j+4],disco1[j+5]);  // primeiro ponto da base de cima
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            colors = colors.concat(r,g,b);
            this.normals = this.normals.concat([disco2[j],disco2[j+1],0]);
            this.normals = this.normals.concat([disco2[j+3],disco2[j+4],0]);
            this.normals = this.normals.concat([disco1[j+3],disco1[j+4],0]);
            
        }
        //console.log(vertices);
        //console.log(colors);
        
        
        // TEXTURE COORDS
        //this.tcoords = [
            //glm.vec2(0.0,0.0),
            //glm.vec2(0.0,1.0),
            //glm.vec2(1.0,0.0)
        //];
        //this.tcoords = [
            //glm.vec2(0.0,1.0),
            //glm.vec2(0.0,0.0),
            //glm.vec2(1.0,0.0)
        //];
        
        //for (var j=0; j<vertices.length/3; j=j+3){
           //this.texCoords = this.texCoords.concat(this.tcoords[0].array); 
           //this.texCoords = this.texCoords.concat(this.tcoords[1].array);
           //this.texCoords = this.texCoords.concat(this.tcoords[2].array);
        //}
        for (var i=1; i<31; i++){
            this.texCoords = this.texCoords.concat(i*1/10-1/10, 1);
            this.texCoords = this.texCoords.concat(i*1/10-1/10, 0);
            this.texCoords = this.texCoords.concat(i*1/10, 0);
            
            this.texCoords = this.texCoords.concat(i*1/10, 1);
            this.texCoords = this.texCoords.concat(i*1/10-1/10, 1);
            this.texCoords = this.texCoords.concat(i*1/10, 0);
        }
        
        
        this.numvertices = vertices.length;
        
        
        // VERTICES
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        // COLORS
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // TEXTURE COORDS
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW); 
        
        // NORMALS
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        var a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
    }
}


class cilindroT extends cilindro{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    }    
}

class cilindroText extends cilindro{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat, local);
    }   
}


</script>

### Balls

In [ ]:
%%html
    
<script id="myapp">

class esfera extends CGRAobject{
    constructor(glcontext, r, g, b){
        super(glcontext); // initialize the parent class
        
        var vertices = [];
        var colors = [];
        this.texCoords = [];
        this.normals = [];
        var disco = [];
        
        // Criar 11 discos com diferentes valores de z
        for (var i = 0; i <= 10; i++){
            disco[i] = [];  
            var raio = 1.0 * Math.cos((Math.PI/10*i) - Math.PI/2);    
            for (var j = 0.0; j <= 2.0*Math.PI; j += 1.0*Math.PI/10){
                disco[i] = disco[i].concat(raio*Math.cos(j),raio*Math.sin(j),Math.sin((Math.PI/10*i) - Math.PI/2));
            }
    
        }
        
        // VERTICES
        // Unir os pontos entre cada par de discos
        for (var i = 0; i < disco.length-1 ; i++){
            for (var j=0; j < disco[0].length; j=j+3){
                vertices = vertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                vertices = vertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto do disco de baixo
                vertices = vertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte mesmo disco
                colors = colors.concat(r,g,b);
                colors = colors.concat(r,g,b);
                colors = colors.concat(r,g,b);
                
                vertices = vertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte disco de cima
                vertices = vertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto disco de cima
                vertices = vertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte do disco de baixo
                colors = colors.concat(r,g,b);
                colors = colors.concat(r,g,b);
                colors = colors.concat(r,g,b);
            }
        }
        
        // TEXT COORDS
        var dimY = 1/10;
        var dimX = 1/(10*2);
        for (var y = 1; y <= 10+1; y++){   // discos
            for (var x = 10*2; x >=0 ; x--){   // pontos dos discos
                this.texCoords = this.texCoords.concat(x*dimX, y*dimY);
                this.texCoords = this.texCoords.concat(x*dimX, y*dimY-dimY);
                this.texCoords = this.texCoords.concat(x*dimX-dimX, y*dimY-dimY);
                    
                this.texCoords = this.texCoords.concat(x*dimX-dimX, y*dimY);
                this.texCoords = this.texCoords.concat(x*dimX, y*dimY);
                this.texCoords = this.texCoords.concat(x*dimX-dimX, y*dimY-dimY);
            }
        }
        
        // NORMALS
        for (var j=0; j < vertices.length; j=j+9){ 
            var aux = [];
            for (var i=0; i < 3; i++){
                aux = aux.concat(vertices[j+i*3], vertices[j+i*3+1], vertices[j+i*3+2])
            }
            
            // Triangle.p2 - Triangle.p1
            var Ux = aux[0]-aux[3];
            var Uy = aux[1]-aux[4];
            var Uz = aux[2]-aux[5];
            
            // Triangle.p3 - Triangle.p1
            var Vx = aux[6]-aux[3];
            var Vy = aux[7]-aux[4];
            var Vz = aux[8]-aux[5];
            
            var U = glm.vec3(Ux,Uy,Uz);
            var V = glm.vec3(Vx,Vy,Vz);

            for (var i=0; i<3; i++){
                this.normals = this.normals.concat(glm.normalize(glm.cross(V,U)).array);
            }
        }
        
        this.numvertices = vertices.length;
        
        // VERTICES
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);
        
        // COLORS
        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // TEXTURE COORDS
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW); 
        
        // NORMALS
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        var a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);      
        
    }
}

class esferaText extends esfera{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    } 
}


class esferaT extends esfera{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    }
}

class esferaL extends esfera{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, lightColor.array);
        
        // LIGHT DIRECTION AND POSITION
        var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = lightPos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(lightPos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    } 
}


</script>

In [ ]:
%%html

<script id="myapp">

class estrela extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 30;
        var vertices = [];
        var disco = [];
        this.texCoords = [];
        this.normals = [];
        
        for (var j = 0.0; j <= 2*Math.PI; j += 2.0*Math.PI/5){   // Array com 33 elementos/pontos
                disco = disco.concat(1.0*Math.cos(j), 1.0*Math.sin(j), -1.0);
        }
        //console.log(disco);
        
                
        // Unir pontos do disco ao centro do cone
        for (var j=0; j<15; j+=3){
            vertices = vertices.concat(disco[j],disco[j+1],disco[j+2]); 
            vertices = vertices.concat(0,0,-1); 
            vertices = vertices.concat(disco[j+3],disco[j+4],disco[j+5]);         
        }
            vertices = vertices.concat(disco[0],disco[1],disco[2]);
            vertices = vertices.concat(2,1,-1);
            vertices = vertices.concat(disco[3],disco[4],disco[5]);
            
            vertices = vertices.concat(disco[3],disco[4],disco[5]);
            vertices = vertices.concat(-1,2,-1);
            vertices = vertices.concat(disco[6],disco[7],disco[8]);
        
             vertices = vertices.concat(disco[6],disco[7],disco[8]);
             vertices = vertices.concat(-2,0,-1);
             vertices = vertices.concat(disco[9],disco[10],disco[11]);
             
             vertices = vertices.concat(disco[9],disco[10],disco[11]);
             vertices = vertices.concat(-1,-2,-1);
             vertices = vertices.concat(disco[12],disco[13],disco[14]);
         
             vertices = vertices.concat(disco[12],disco[13],disco[14]);
             vertices = vertices.concat(2,-1,-1);
             vertices = vertices.concat(disco[15],disco[16],disco[17]);

        var colors = [
            1.0, 1.0, 0.0, 
            1.0, 1.0, 0.0,
            1.0, 1.0, 0.0];
        
        for (var i=0; i < 12; i++){
            colors = colors.concat(colors);
        }
        //console.log(colors);
        
        
        this.tcoords = [
            glm.vec2(0.0,1.0),
            glm.vec2(0.0,0.0),
            glm.vec2(1.0,0.0)
        ];
        
        for (var j=0; j<vertices.length/3; j=j+3){
           this.texCoords = this.texCoords.concat(this.tcoords[0].array); 
           this.texCoords = this.texCoords.concat(this.tcoords[1].array);
           this.texCoords = this.texCoords.concat(this.tcoords[2].array);
        }
        
        for (var j=0; j<vertices.length/3; j=j+3){
            this.normals = this.normals.concat([0,1,0]);
            this.normals = this.normals.concat([0,1,0]);
            this.normals = this.normals.concat([0,1,0]);
        }
        
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
         // TEXTURE COORDS
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW); 
        
        // NORMALS
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        var a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    }
}

class estrelaT extends estrela{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    }  
}

</script>

In [ ]:
%%html
<canvas id="myCanvas1" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>

<script id="myapp">

class myapp1 extends DEECapp{
    counter=0;
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        var fragsrcL = document.getElementById("my-fragment-shader-light").text;
        var vertsrcL = document.getElementById("my-vertex-shader-light").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        var fragsrcD = document.getElementById("my-fragment-shader-D").text;
        var vertsrcD = document.getElementById("my-vertex-shader-D").text;
        
        this.shaderprogD = new DEECshader(this.gl);
        this.shaderprogD.srcShaders(vertsrcD,fragsrcD);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        //this.gl.clearColor(0.30,0.30,0.30,1.0);
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,1,100); 
        this.viewM = glm.lookAt(glm.vec3(0,0,0),glm.vec3(0,0,-1),glm.vec3(0,1,0));
        
        // Lets create one cone, one cylinder and sphere objects 
        this.estrela = new estrelaT(this.gl);
        this.estrela.setViewPos(glm.vec3(0,0,0));
        this.estrela.setLights(glm.vec3(0,100,0), glm.vec3(1.0,1.0,1.0));
        this.estrela.setShader(this.shaderprogD);
        this.mycone = new coneText(this.gl);
        this.mycone.setShader(this.shaderprogT);
        this.mycone2 = new coneText(this.gl);
        this.mycone2.setShader(this.shaderprogT);
        this.mycone3 = new coneText(this.gl);
        this.mycone3.setShader(this.shaderprogT);
        this.mycilindro = new cilindroText(this.gl);
        this.mycilindro.setShader(this.shaderprogT);
        
        this.myesfera = new esferaL(this.gl,0.0,0.0,1.0);
        this.myesfera.setShader(this.shaderprogL);
        this.myesfera1 = new esferaL(this.gl,1.0,0.0,0.0);
        this.myesfera1.setShader(this.shaderprogL);
        this.myesfera2 = new esferaL(this.gl,0.9,0.6,0.3);
        this.myesfera2.setShader(this.shaderprogL); 
        this.myesfera3 = new esferaL(this.gl,0.2,0.0,0.7);
        this.myesfera3.setShader(this.shaderprogL); 
        this.myesfera4 = new esferaL(this.gl,0.7,0.5,1.0);
        this.myesfera4.setShader(this.shaderprogL);
        this.myesfera5 = new esferaL(this.gl,0.5,0.6,0.9);
        this.myesfera5.setShader(this.shaderprogL);
        this.myesfera6 = new esferaL(this.gl,0.9,0.5,0.0);
        this.myesfera6.setShader(this.shaderprogL);
        this.myesfera7 = new esferaL(this.gl,0.7,0.0,0.5);
        this.myesfera7.setShader(this.shaderprogL);
        this.myesfera8 = new esferaL(this.gl,0.2,0.8,0.3);
        this.myesfera8.setShader(this.shaderprogL);
        this.myesfera9 = new esferaL(this.gl,0.1,0.5,0.8);
        this.myesfera9.setShader(this.shaderprogL);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("greenish_texture.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("brown_texture.jpg");
        this.texture3 = new CGRAtexture(this.gl);
        this.texture3.load("red_texture.jpg");
        this.texture4 = new CGRAtexture(this.gl);
        this.texture4.load("textura_amarela.jpg");
        
        this.estrela.settexture(this.texture4);
        this.mycone.settexture(this.texture);
        this.mycone2.settexture(this.texture);
        this.mycone3.settexture(this.texture);
        this.mycilindro.settexture(this.texture2);
        this.myesfera.settexture(this.texture3);
        this.myesfera1.settexture(this.texture3);
        this.myesfera2.settexture(this.texture3);
        this.myesfera3.settexture(this.texture3);
        this.myesfera4.settexture(this.texture3);
        this.myesfera5.settexture(this.texture3);
        this.myesfera6.settexture(this.texture3);
        this.myesfera7.settexture(this.texture3);
        this.myesfera8.settexture(this.texture3);
        this.myesfera9.settexture(this.texture3);
    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);

        this.counter++;
        
        // define the model tranformation for estrela
        var scale = glm.mat4(glm.mat3(0.05));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(-15),glm.vec3(0.0,0.0,1.0)));
        //rotation = glm.mat4(0.1)['*'](rotation['*'](scale));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.85,0.0));
        rotation = transl['*'](rotation['*'](scale));
        this.estrela.setModelTransformation(rotation);
        
        // define the model tranformation for the first
        var scale = glm.mat4(glm.mat3(0.4));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(-90),glm.vec3(1.0,0.0,0.0)));
        //rotation = glm.mat4(0.1)['*'](rotation['*'](scale));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.5,0.0));
        rotation = transl['*'](rotation['*'](scale));
        this.mycone.setModelTransformation(rotation);
        
        var scale = glm.mat4(glm.mat3(0.5));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(-90),glm.vec3(1.0,0.0,0.0)));
        //rotation = glm.mat4(0.1)['*'](rotation['*'](scale));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.25,0.0));
        rotation = transl['*'](rotation['*'](scale));
        this.mycone2.setModelTransformation(rotation);
        
        var scale = glm.mat4(glm.mat3(0.6));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(-90),glm.vec3(1.0,0.0,0.0)));
        //rotation = glm.mat4(0.1)['*'](rotation['*'](scale));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.0,0.0));
        rotation = transl['*'](rotation['*'](scale));
        this.mycone3.setModelTransformation(rotation);
       
        // define the model tranformation for the second
        var scale = glm.scale(glm.vec3(0.1,0.1,0.5));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,-0.5,0.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.mycilindro.setModelTransformation(transformation);
        
        // define the model tranformation for the spheres
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(-0.3,0.15,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.3,0.15,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera1.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.65,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera2.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,0.35,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera3.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(-0.4,-0.2,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera4.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.4,-0.2,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera5.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(-0.45,-0.5,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera6.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.45,-0.5,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera7.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,-0.0,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera8.setModelTransformation(transformation);
        
        var scale = glm.scale(glm.vec3(0.05,0.05,0.05));
        rotation = glm.rotate(glm.mat4(1.0),glm.radians(90),glm.vec3(1.0,.0,0.0));
        var transl = glm.translate(glm.mat4(1.0),glm.vec3(0.0,-0.4,-1.0));
        var transformation = transl['*'](rotation['*'](scale));
        this.myesfera9.setModelTransformation(transformation);
        
        // draw
        this.estrela.drawit();
        this.mycone.drawit();
        this.mycone2.drawit();
        this.mycone3.drawit();
        this.mycilindro.drawit();
        this.myesfera.setViewPos(glm.vec3(0,0,0));
        this.myesfera.drawit();
        this.myesfera1.setViewPos(glm.vec3(0,0,0));
        this.myesfera1.drawit();
        this.myesfera2.setViewPos(glm.vec3(0,0,0));
        this.myesfera2.drawit();
        this.myesfera3.setViewPos(glm.vec3(0,0,0));
        this.myesfera3.drawit();
        this.myesfera4.setViewPos(glm.vec3(0,0,0));
        this.myesfera4.drawit();
        this.myesfera5.setViewPos(glm.vec3(0,0,0));
        this.myesfera5.drawit();
        this.myesfera6.setViewPos(glm.vec3(0,0,0));
        this.myesfera6.drawit();
        this.myesfera7.setViewPos(glm.vec3(0,0,0));
        this.myesfera7.drawit();
        this.myesfera8.setViewPos(glm.vec3(0,0,0));
        this.myesfera8.drawit();
        this.myesfera9.setViewPos(glm.vec3(0,0,0));
        this.myesfera9.drawit();
    }
}

var app1 = new myapp1('myCanvas1');

app1.run();

</script>

In [ ]:
%%html
    
<canvas id="myCanvas2" width="400" height="400" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>    
    
<script id="myapp">

class myapp2 extends DEECapp{
    counter=0;
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        //this.gl.clearColor(0.30,0.30,0.30,1.0);
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,1,100); 
        this.viewM = glm.lookAt(glm.vec3(0,0,0),glm.vec3(0,0,-1),glm.vec3(0,1,0));
        
        
        // Lets create one esfera 
        this.myesfera = new esferaText(this.gl);
        this.myesfera.setShader(this.shaderprogT);
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("orange_texture.jpg");
        
        this.myesfera.settexture(this.texture);

    }
    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);

        this.counter++;
        
        // define the model tranformation
        var scale = glm.mat4(glm.mat3(1));
        var rotation = glm.toMat4(glm.angleAxis(glm.radians(this.counter),glm.vec3(1.0,0.0,0.0)));
        rotation = glm.mat4(0.1)['*'](rotation['*'](scale));
        rotation = (rotation['*'](scale));
        this.myesfera.setModelTransformation(rotation);
        
        // draw
        this.myesfera.drawit();

    }
}

var app2 = new myapp2('myCanvas2');

app2.run();

</script>

In [ ]:
%%html
    
<script id="myapp">

class compoundObjectCar1 extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
            
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("red_texture.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("wheel_texture.jpg");
        
        this.roda1 = new cilindroT(this.gl);
        this.roda1.settexture(this.texture2);
        this.roda2 = new cilindroT(this.gl);
        this.roda2.settexture(this.texture2);
        this.roda3 = new cilindroT(this.gl);
        this.roda3.settexture(this.texture2);
        this.roda4 = new cilindroT(this.gl);
        this.roda4.settexture(this.texture2);
        
        this.base = new cubeT(this.gl);
        this.base.settexture(this.texture);
        this.habitaculo = new cubeT(this.gl);
        this.habitaculo.settexture(this.texture);
               
        var rotate90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(0.0,1.0,0.0)));
        var positionBase = glm.vec3(0,1,0);
        
        // Base
        scale = glm.scale(glm.vec3(5.5,1.0,6.5));
        var transformation = glm.translate(glm.vec3(positionBase[0],positionBase[1]+0.3,positionBase[2]));
        var transformation2 = transformation['*'](scale);
        this.base.setModelTransformation(transformation['*'](scale));
        
        // Habitaculo
        scale = glm.scale(glm.vec3(3.5,2.5,3));
        transformation = glm.translate(glm.vec3(positionBase[0],positionBase[1]+1.5,positionBase[2]));
        transformation2 = transformation['*'](scale);
        this.habitaculo.setModelTransformation(transformation2);
        
        // Roda 1
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]-2,positionBase[1]-1,positionBase[2]-2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda1.setModelTransformation(transformation2);
        
        // Roda 2
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]+2,positionBase[1]-1,positionBase[2]-2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda2.setModelTransformation(transformation2);
        
        // Roda 3
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]+2,positionBase[1]-1,positionBase[2]+2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda3.setModelTransformation(transformation2);
        
        // Roda 4
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]-2,positionBase[1]-1,positionBase[2]+2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda4.setModelTransformation(transformation2);   

     }
        
    
    // novo método setShader() para associar o shader para cada um dos objetos
    
    setShader(shader){
        this.roda1.setShader(shader);
        this.roda2.setShader(shader); 
        this.roda3.setShader(shader); 
        this.roda4.setShader(shader); 
        this.base.setShader(shader);
        this.habitaculo.setShader(shader);
    } 
    
    setViewPos(viewPos){
        this.roda1.setViewPos(viewPos);
        this.roda2.setViewPos(viewPos); 
        this.roda3.setViewPos(viewPos); 
        this.roda4.setViewPos(viewPos); 
        this.base.setViewPos(viewPos);
        this.habitaculo.setViewPos(viewPos);
    }
    
    setLights(lightposit, lightcor){
        this.roda1.setLights(lightposit, lightcor);
        this.roda2.setLights(lightposit, lightcor);
        this.roda3.setLights(lightposit, lightcor);
        this.roda4.setLights(lightposit, lightcor);
        this.base.setLights(lightposit, lightcor);
        this.habitaculo.setLights(lightposit, lightcor);
    }
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        //console.log(local.array);
        
        this.roda1.drawit(viewMat, projectionMat, local);
        this.roda2.drawit(viewMat, projectionMat, local);
        this.roda3.drawit(viewMat, projectionMat, local);
        this.roda4.drawit(viewMat, projectionMat, local);
        this.base.drawit(viewMat, projectionMat, local);
        this.habitaculo.drawit(viewMat, projectionMat, local);            
    } 
    
    
}


class compoundObjectCar2 extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("blue_texture.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("wheel_texture.jpg");        
        
        
        this.roda1 = new cilindroT(this.gl);
        this.roda1.settexture(this.texture2);
        this.roda2 = new cilindroT(this.gl);
        this.roda2.settexture(this.texture2);
        this.roda3 = new cilindroT(this.gl);
        this.roda3.settexture(this.texture2);
        this.roda4 = new cilindroT(this.gl);
        this.roda4.settexture(this.texture2);
        this.base = new cubeT(this.gl);
        this.base.settexture(this.texture);
        this.habitaculo = new cubeT(this.gl);
        this.habitaculo.settexture(this.texture);
      
        var rotate90 = glm.toMat4(glm.angleAxis(Math.PI/2.0,glm.vec3(0.0,1.0,0.0)));
        var positionBase = glm.vec3(0,1,0);
        
        // Base
        scale = glm.scale(glm.vec3(5.5,1.0,6.5));
        var transformation = glm.translate(glm.vec3(positionBase[0],positionBase[1]+0.3,positionBase[2]));
        var transformation2 = transformation['*'](scale);
        this.base.setModelTransformation(transformation['*'](scale));
        
        // Habitaculo
        scale = glm.scale(glm.vec3(3.5,2.5,3));
        transformation = glm.translate(glm.vec3(positionBase[0],positionBase[1]+1.5,positionBase[2]));
        transformation2 = transformation['*'](scale);
        this.habitaculo.setModelTransformation(transformation2);
        
        // Roda 1
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]-2,positionBase[1]-1,positionBase[2]-2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda1.setModelTransformation(transformation2);
        
        // Roda 2
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]+2,positionBase[1]-1,positionBase[2]-2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda2.setModelTransformation(transformation2);
        
        // Roda 3
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]+2,positionBase[1]-1,positionBase[2]+2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda3.setModelTransformation(transformation2);
        
        // Roda 4
        var scale = glm.scale(glm.vec3(1,1,0.5));
        var transformation = glm.translate(glm.vec3(positionBase[0]-2,positionBase[1]-1,positionBase[2]+2));
        transformation2 = transformation['*'](rotate90)['*'](scale);
        this.roda4.setModelTransformation(transformation2);
        
               
     }
        
    
    // novo método setShader() para associar o shader para cada um dos objetos
    setShader(shader){
        this.roda1.setShader(shader);
        this.roda2.setShader(shader); 
        this.roda3.setShader(shader); 
        this.roda4.setShader(shader); 
        this.base.setShader(shader);
        this.habitaculo.setShader(shader);
    }
    
    setViewPos(viewPos){
        this.roda1.setViewPos(viewPos);
        this.roda2.setViewPos(viewPos); 
        this.roda3.setViewPos(viewPos); 
        this.roda4.setViewPos(viewPos); 
        this.base.setViewPos(viewPos);
        this.habitaculo.setViewPos(viewPos);
    }
    
    setLights(lightposit, lightcor){
        this.roda1.setLights(lightposit, lightcor);
        this.roda2.setLights(lightposit, lightcor);
        this.roda3.setLights(lightposit, lightcor);
        this.roda4.setLights(lightposit, lightcor);
        this.base.setLights(lightposit, lightcor);
        this.habitaculo.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.roda1.drawit(viewMat, projectionMat, local);
        this.roda2.drawit(viewMat, projectionMat, local);
        this.roda3.drawit(viewMat, projectionMat, local);
        this.roda4.drawit(viewMat, projectionMat, local);
        this.base.drawit(viewMat, projectionMat, local);
        this.habitaculo.drawit(viewMat, projectionMat, local);            
    } 
    
    
}

class compoundObjectArvore extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("greenish_texture.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("brown_texture.jpg");
             
        this.tronco1 = new cilindroT(this.gl);
        this.tronco1.settexture(this.texture2);
        this.tronco2 = new cilindroT(this.gl);
        this.tronco2.settexture(this.texture2);
        this.topo1 = new coneT(this.gl);
        this.topo1.settexture(this.texture);
        this.topo2 = new coneT(this.gl);
        this.topo2.settexture(this.texture);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Tronco1 -> Cilindro
        var scale = glm.scale(glm.vec3(0.7,0.7,8));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1],troncoPos[2]));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.tronco1.setModelTransformation(transformation2);
          
        // Topo1 -> Cone
        var scale = glm.scale(glm.vec3(4,4,4));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1]+6,troncoPos[2]));
        this.topo1.setModelTransformation(transformation['*'](scale));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo1.setModelTransformation(transformation2);
        
        // Tronco2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.7,0.7,8));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1],troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.tronco2.setModelTransformation(transformation2);
        
        // Topo2 -> Cone
        var scale = glm.scale(glm.vec3(4,4,4));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1]+6,troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo2.setModelTransformation(transformation2);        
     }
        
    
    // novo método setShader() para associar o shader para cada um dos objetos
    
    setShader(shader){
        this.tronco1.setShader(shader);
        this.topo1.setShader(shader); 
        this.tronco2.setShader(shader); 
        this.topo2.setShader(shader); 
    }
    
    setViewPos(viewPos){
        this.tronco1.setViewPos(viewPos);
        this.topo1.setViewPos(viewPos); 
        this.tronco2.setViewPos(viewPos); 
        this.topo2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.tronco1.setLights(lightposit, lightcor);
        this.topo1.setLights(lightposit, lightcor);
        this.tronco2.setLights(lightposit, lightcor);
        this.topo2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.tronco1.drawit(viewMat, projectionMat, local);
        this.topo1.drawit(viewMat, projectionMat, local);
        this.tronco2.drawit(viewMat, projectionMat, local);
        this.topo2.drawit(viewMat, projectionMat, local);          
    } 
    
    
}


class compoundObjectArvoreNatal extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.estrela1 = new estrelaT(this.gl);
        this.estrela2 = new estrelaT(this.gl);
        this.tronco1 = new cilindroT(this.gl);
        this.tronco2 = new cilindroT(this.gl);
        this.topo1 = new coneT(this.gl);
        this.topo1_2 = new coneT(this.gl);
        this.topo1_3 = new coneT(this.gl);
        this.topo2 = new coneT(this.gl);
        this.topo2_2 = new coneT(this.gl);
        this.topo2_3 = new coneT(this.gl);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var rotate90y = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(0.0,1.0,0.0)));
        var rotate45y = glm.toMat4(glm.angleAxis(-Math.PI/4.0,glm.vec3(0.0,1.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Tronco1 -> Cilindro
        var scale = glm.scale(glm.vec3(0.7,0.7,8));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1],troncoPos[2]));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.tronco1.setModelTransformation(transformation2);
          
        // Topo1 -> Cone
        var scale = glm.scale(glm.vec3(4,4,4));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1]+6,troncoPos[2]));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo1.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(5,5,5));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1]+4,troncoPos[2]));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo1_2.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(5.5,5.5,5.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1]+2,troncoPos[2]));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo1_3.setModelTransformation(transformation2);
        
        // Tronco2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.7,0.7,8));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1],troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.tronco2.setModelTransformation(transformation2);
        
        // Topo2 -> Cone
        var scale = glm.scale(glm.vec3(4,4,4));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1]+6,troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo2.setModelTransformation(transformation2);  
        
        var scale = glm.scale(glm.vec3(5,5,5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1]+4,troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo2_2.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(5.5,5.5,5.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1]+2,troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo2_3.setModelTransformation(transformation2);
        
        // Estrela 1
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0],troncoPos[1]+10,troncoPos[2]));
        var transformation2 = transformation['*'](rotate90y)['*'](scale)
        this.estrela1.setModelTransformation(transformation2);
        
        // Estrela 2
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+20,troncoPos[1]+10,troncoPos[2]-20));
        var transformation2 = transformation['*'](rotate45y)['*'](scale)
        this.estrela2.setModelTransformation(transformation2);
        
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("greenish_texture.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("brown_texture.jpg");
        this.texture3 = new CGRAtexture(this.gl);
        this.texture3.load("red_texture.jpg");
        this.texture4 = new CGRAtexture(this.gl);
        this.texture4.load("textura_amarela.jpg");
        
        this.topo1.settexture(this.texture);
        this.topo1_2.settexture(this.texture);
        this.topo1_3.settexture(this.texture);
        this.topo2.settexture(this.texture);
        this.topo2_2.settexture(this.texture);
        this.topo2_3.settexture(this.texture);
        this.tronco1.settexture(this.texture2);
        this.tronco2.settexture(this.texture2);
        this.estrela1.settexture(this.texture4);
        this.estrela2.settexture(this.texture4);
        
        // Bolas -> Esferas
        this.myesfera = new esferaT(this.gl);
        this.myesfera1 = new esferaT(this.gl);
        this.myesfera2 = new esferaT(this.gl);
        this.myesfera3 = new esferaT(this.gl);
        
        this.myesfera4 = new esferaT(this.gl);
        this.myesfera5 = new esferaT(this.gl);
        this.myesfera6 = new esferaT(this.gl);
        this.myesfera7 = new esferaT(this.gl);
        
        this.myesfera8 = new esferaT(this.gl);
        this.myesfera9 = new esferaT(this.gl);
        this.myesfera10 = new esferaT(this.gl);
        this.myesfera11 = new esferaT(this.gl);
        
        this.myesfera12 = new esferaT(this.gl);
        this.myesfera13 = new esferaT(this.gl);
        this.myesfera14 = new esferaT(this.gl);
        this.myesfera15 = new esferaT(this.gl);
        
        this.myesfera16 = new esferaT(this.gl);
        this.myesfera17 = new esferaT(this.gl);
        this.myesfera18 = new esferaT(this.gl);
        this.myesfera19 = new esferaT(this.gl);
        
        
        this.myesfera.settexture(this.texture3);
        this.myesfera1.settexture(this.texture3);
        this.myesfera2.settexture(this.texture3);
        this.myesfera3.settexture(this.texture3);
        
        this.myesfera4.settexture(this.texture3);
        this.myesfera5.settexture(this.texture3);
        this.myesfera6.settexture(this.texture3);
        this.myesfera7.settexture(this.texture3);
        
        this.myesfera8.settexture(this.texture3);
        this.myesfera9.settexture(this.texture3);
        this.myesfera10.settexture(this.texture3);
        this.myesfera11.settexture(this.texture3);
        
        this.myesfera12.settexture(this.texture3);
        this.myesfera13.settexture(this.texture3);
        this.myesfera14.settexture(this.texture3);
        this.myesfera15.settexture(this.texture3);
        

        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+6,troncoPos[2]-1.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+6,troncoPos[2]-1.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera1.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+2.5,troncoPos[2]-3.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera2.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+2.5,troncoPos[2]-3.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera3.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+2.5,troncoPos[2]+3.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera4.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+2.5,troncoPos[2]+3.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera5.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+6,troncoPos[2]+1.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera6.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+6,troncoPos[2]+1.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera7.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+0.5,troncoPos[2]+4.1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera8.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]+0.5,troncoPos[2]-4.1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera9.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+0.5,troncoPos[2]-4.1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera10.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]+0.5,troncoPos[2]+4.1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera11.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]-1.5,troncoPos[2]-4.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera12.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+2,troncoPos[1]-1.5,troncoPos[2]+4.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera13.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]-1.5,troncoPos[2]-4.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera14.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-2,troncoPos[1]-1.5,troncoPos[2]+4.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.myesfera15.setModelTransformation(transformation2);
        
       
        
    
     }
        
    
    // novo método setShader() para associar o shader para cada um dos objetos
    
    setShader(shader){
        this.tronco1.setShader(shader);
        this.topo1.setShader(shader); 
        this.topo1_2.setShader(shader); 
        this.topo1_3.setShader(shader); 
        this.tronco2.setShader(shader); 
        this.topo2.setShader(shader);
        this.topo2_2.setShader(shader);
        this.topo2_3.setShader(shader);
       
        this.myesfera.setShader(shader);
        this.myesfera1.setShader(shader); 
        this.myesfera2.setShader(shader); 
        this.myesfera3.setShader(shader);
        
         this.myesfera4.setShader(shader);
        this.myesfera5.setShader(shader); 
        this.myesfera6.setShader(shader); 
        this.myesfera7.setShader(shader);
        
         this.myesfera8.setShader(shader);
        this.myesfera9.setShader(shader); 
        this.myesfera10.setShader(shader); 
        this.myesfera11.setShader(shader);
        
         this.myesfera12.setShader(shader);
        this.myesfera13.setShader(shader); 
        this.myesfera14.setShader(shader); 
        this.myesfera15.setShader(shader);
        
      
        this.estrela1.setShader(shader); 
        this.estrela2.setShader(shader);
    }
    
    setViewPos(viewPos){
        this.tronco1.setViewPos(viewPos);
        this.topo1.setViewPos(viewPos); 
        this.topo1_2.setViewPos(viewPos); 
        this.topo1_3.setViewPos(viewPos); 
        this.tronco2.setViewPos(viewPos);
        this.topo2.setViewPos(viewPos); 
        this.topo2_2.setViewPos(viewPos); 
        this.topo2_3.setViewPos(viewPos);
        this.myesfera.setViewPos(viewPos);
        this.myesfera1.setViewPos(viewPos); 
        this.myesfera2.setViewPos(viewPos); 
        this.myesfera3.setViewPos(viewPos);
        
        this.myesfera4.setViewPos(viewPos);
        this.myesfera5.setViewPos(viewPos); 
        this.myesfera6.setViewPos(viewPos); 
        this.myesfera7.setViewPos(viewPos);
        
        this.myesfera8.setViewPos(viewPos);
        this.myesfera9.setViewPos(viewPos); 
        this.myesfera10.setViewPos(viewPos); 
        this.myesfera11.setViewPos(viewPos);
        
        this.myesfera12.setViewPos(viewPos);
        this.myesfera13.setViewPos(viewPos); 
        this.myesfera14.setViewPos(viewPos); 
        this.myesfera15.setViewPos(viewPos);
        
  
        
        this.estrela1.setViewPos(viewPos); 
        this.estrela2.setViewPos(viewPos);
    }
    
    setLights(lightposit, lightcor){
        this.tronco1.setLights(lightposit, lightcor);
        this.topo1.setLights(lightposit, lightcor);
        this.topo1_2.setLights(lightposit, lightcor);
        this.topo1_3.setLights(lightposit, lightcor);
        this.tronco2.setLights(lightposit, lightcor);
        this.topo2.setLights(lightposit, lightcor);
        this.topo2_2.setLights(lightposit, lightcor);
        this.topo2_3.setLights(lightposit, lightcor);
        this.myesfera.setLights(lightposit, lightcor);
        this.myesfera1.setLights(lightposit, lightcor);
        this.myesfera2.setLights(lightposit, lightcor);
        this.myesfera3.setLights(lightposit, lightcor);
        
        this.myesfera4.setLights(lightposit, lightcor);
        this.myesfera5.setLights(lightposit, lightcor);
        this.myesfera6.setLights(lightposit, lightcor);
        this.myesfera7.setLights(lightposit, lightcor);
        
        this.myesfera8.setLights(lightposit, lightcor);
        this.myesfera9.setLights(lightposit, lightcor);
        this.myesfera10.setLights(lightposit, lightcor);
        this.myesfera11.setLights(lightposit, lightcor);
        
        this.myesfera12.setLights(lightposit, lightcor);
        this.myesfera13.setLights(lightposit, lightcor);
        this.myesfera14.setLights(lightposit, lightcor);
        this.myesfera15.setLights(lightposit, lightcor);
        
        this.estrela1.setLights(lightposit, lightcor);
        this.estrela2.setLights(lightposit, lightcor);
      
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.tronco1.drawit(viewMat, projectionMat, local);
        this.topo1.drawit(viewMat, projectionMat, local);
        this.topo1_2.drawit(viewMat, projectionMat, local);
        this.topo1_3.drawit(viewMat, projectionMat, local);
        this.tronco2.drawit(viewMat, projectionMat, local);
        this.topo2.drawit(viewMat, projectionMat, local); 
        this.topo2_2.drawit(viewMat, projectionMat, local);
        this.topo2_3.drawit(viewMat, projectionMat, local);
        this.myesfera.drawit(viewMat, projectionMat, local);
        this.myesfera1.drawit(viewMat, projectionMat, local);
        this.myesfera2.drawit(viewMat, projectionMat, local);
        this.myesfera3.drawit(viewMat, projectionMat, local);
        
        this.myesfera4.drawit(viewMat, projectionMat, local);
        this.myesfera5.drawit(viewMat, projectionMat, local);
        this.myesfera6.drawit(viewMat, projectionMat, local);
        this.myesfera7.drawit(viewMat, projectionMat, local);
        
        this.myesfera8.drawit(viewMat, projectionMat, local);
        this.myesfera9.drawit(viewMat, projectionMat, local);
        this.myesfera10.drawit(viewMat, projectionMat, local);
        this.myesfera11.drawit(viewMat, projectionMat, local);
        
        this.myesfera12.drawit(viewMat, projectionMat, local);
        this.myesfera13.drawit(viewMat, projectionMat, local);
        this.myesfera14.drawit(viewMat, projectionMat, local);
        this.myesfera15.drawit(viewMat, projectionMat, local);
        
      
        
        this.estrela1.drawit(viewMat, projectionMat, local);
        this.estrela2.drawit(viewMat, projectionMat, local);
    } 
    
    
}


class compoundObjectBoneco extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("nuvem_textura.png");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("orange_texture.jpg");
        this.texture3 = new CGRAtexture(this.gl);
        this.texture3.load("textura_preta.jpg");
           
        this.tronco1 = new esferaT(this.gl);
        this.tronco1.settexture(this.texture);
        this.olho1 = new esferaT(this.gl);
        this.olho1.settexture(this.texture3);
        this.olho2 = new esferaT(this.gl);
        this.olho2.settexture(this.texture3);
        this.topo1 = new esferaT(this.gl);
        this.topo1.settexture(this.texture);
        this.nariz = new coneT(this.gl);
        this.nariz.settexture(this.texture2);
    
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(0.0,0.0,0.0)));
        var rotate1 = glm.toMat4(glm.angleAxis(-Math.PI/1.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,10.0,0.0);
        
        // Tronco1 -> Cilindro
        var scale = glm.scale(glm.vec3(7,7,7));
        var transformation = glm.translate(glm.vec3(-15,troncoPos[1]-8,troncoPos[2]+60));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.tronco1.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(-16,12,56.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.olho1.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(0.5,0.5,0.5));
        var transformation = glm.translate(glm.vec3(-14,12,56.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.olho2.setModelTransformation(transformation2);
          
          
        // Topo1 -> Cone
        var scale = glm.scale(glm.vec3(4,4,4));
        var transformation = glm.translate(glm.vec3(-15,troncoPos[1]+1,troncoPos[2]+60));
        this.topo1.setModelTransformation(transformation['*'](scale));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.topo1.setModelTransformation(transformation2);
        
        var scale = glm.scale(glm.vec3(1,1,2.5));
        var transformation = glm.translate(glm.vec3(-15,troncoPos[1]+1,56));
       // this.nariz.setModelTransformation(transformation['*'](scale));
        var transformation2 = transformation['*'](rotate1)['*'](scale)
        this.nariz.setModelTransformation(transformation2);
                  
     }
        
    
    // novo método setShader() para associar o shader para cada um dos objetos
    
    setShader(shader){
        this.tronco1.setShader(shader);
        this.topo1.setShader(shader); 
        this.olho1.setShader(shader); 
        this.olho2.setShader(shader);
        this.nariz.setShader(shader);
    }
    
    setViewPos(viewPos){
        this.tronco1.setViewPos(viewPos);
        this.topo1.setViewPos(viewPos); 
        this.olho1.setViewPos(viewPos); 
        this.olho2.setViewPos(viewPos); 
        this.nariz.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.tronco1.setLights(lightposit, lightcor);
        this.topo1.setLights(lightposit, lightcor);
        this.olho1.setLights(lightposit, lightcor);
        this.olho2.setLights(lightposit, lightcor);
        this.nariz.setLights(lightposit, lightcor);
    }
    
    
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.tronco1.drawit(viewMat, projectionMat, local);
        this.topo1.drawit(viewMat, projectionMat, local);
        this.olho1.drawit(viewMat, projectionMat, local);
        this.olho2.drawit(viewMat, projectionMat, local);
        this.nariz.drawit(viewMat, projectionMat, local);
        
        
    } 
       
}




class compoundObjectPosteOff extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("nuvem_textura.png");
           
        this.poste1 = new cilindroT(this.gl);
        this.poste1.settexture(this.texture);
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo1 = new esferaT(this.gl);
        this.topo1.settexture(this.texture2);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
              
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste1 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-25,troncoPos[1],troncoPos[2]+18));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste1.setModelTransformation(transformation2);
          
        // Topo1 -> ESFERA
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-25,troncoPos[1]+6,troncoPos[2]+18));
        this.topo1.setModelTransformation(transformation['*'](scale));
        var transformation2 = transformation['*'](scale)
        this.topo1.setModelTransformation(transformation2);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2);  
               
     }
        
    setShader(shader){
        this.topo1.setShader(shader); 
        this.topo2.setShader(shader);
        this.poste1.setShader(shader);
        this.poste2.setShader(shader); 
    }  
    
    setViewPos(viewPos){
        this.topo1.setViewPos(viewPos); 
        this.topo2.setViewPos(viewPos); 
        this.poste1.setViewPos(viewPos); 
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo1.setLights(lightposit, lightcor);
        this.topo2.setLights(lightposit, lightcor);
        this.poste1.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo1.drawit(viewMat, projectionMat, local);
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste1.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}

class compoundObjectPosteOff2 extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("textura_preta.jpg");
           
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
              
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2);  
               
     }
        
    setShader(shader){
        this.topo2.setShader(shader);
        this.poste2.setShader(shader); 
    }  
    
    setViewPos(viewPos){
        this.topo2.setViewPos(viewPos); 
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo2.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}

class compoundObjectPosteW extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("nuvem_textura.png");
           
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2);  
               
     }
        
    setShader(shader){ 
        this.topo2.setShader(shader);
        this.poste2.setShader(shader); 
    }
    
    
    setViewPos(viewPos){
        this.topo2.setViewPos(viewPos);  
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo2.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}

class compoundObjectPosteR extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("red_texture.jpg");
           
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2); 
               
     }
        
    setShader(shader){
        this.topo2.setShader(shader);
        this.poste2.setShader(shader); 
    } 
    
    
    setViewPos(viewPos){
        this.topo2.setViewPos(viewPos); 
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo2.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}

class compoundObjectPosteG extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("greenish_texture.jpg");
           
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2);
     }
        
    setShader(shader){
        this.topo2.setShader(shader);
        this.poste2.setShader(shader); 
    } 
    
    setViewPos(viewPos){ 
        this.topo2.setViewPos(viewPos); 
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo2.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}

class compoundObjectPosteB extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
             
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("blue_texture.jpg");
           
        this.poste2 = new cilindroT(this.gl);
        this.poste2.settexture(this.texture);
        this.topo2 = new esferaT(this.gl);
        this.topo2.settexture(this.texture2);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste2 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1],troncoPos[2]-1));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste2.setModelTransformation(transformation2);
        
        // Topo2 -> Esfera
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]+27,troncoPos[1]+6,troncoPos[2]-1));
        var transformation2 = transformation['*'](scale)
        this.topo2.setModelTransformation(transformation2);  
               
     }
        
    setShader(shader){
        this.topo2.setShader(shader);
        this.poste2.setShader(shader); 
    }
    
    setViewPos(viewPos){
        this.topo2.setViewPos(viewPos); 
        this.poste2.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo2.setLights(lightposit, lightcor);
        this.poste2.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo2.drawit(viewMat, projectionMat, local);
        this.poste2.drawit(viewMat, projectionMat, local);
    } 
    
    
}


class compoundObjectPosteBranco extends CGRAobject {
      
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.texture2 = new CGRAtexture(this.gl);
        this.texture2.load("nuvem_textura.png");
           
        this.poste1 = new cilindroT(this.gl);
        this.poste1.settexture(this.texture);
        this.topo1 = new esferaT(this.gl);
        this.topo1.settexture(this.texture2);
              
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var troncoPos = glm.vec3(0.0,7.0,0.0);
        
        // Poste1 -> Cilindro
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-25,troncoPos[1],troncoPos[2]+18));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poste1.setModelTransformation(transformation2);
          
        // Topo1 -> ESFERA
        var scale = glm.scale(glm.vec3(2,2,2));
        var transformation = glm.translate(glm.vec3(troncoPos[0]-25,troncoPos[1]+6,troncoPos[2]+18));
        this.topo1.setModelTransformation(transformation['*'](scale));
        var transformation2 = transformation['*'](scale)
        this.topo1.setModelTransformation(transformation2); 
               
     }
        
    setShader(shader){
        this.topo1.setShader(shader); 
        this.poste1.setShader(shader);
    }  
    
    setViewPos(viewPos){
        this.topo1.setViewPos(viewPos); 
        this.poste1.setViewPos(viewPos); 
    }
    
    setLights(lightposit, lightcor){
        this.topo1.setLights(lightposit, lightcor);
        this.poste1.setLights(lightposit, lightcor);
    }
        
    
    // novo método drawit() para desenhar cada um dos objetos 
    
    drawit(viewMat, projectionMat, parentMat = glm.mat4(1.0)){
        
        var local = parentMat['*'](this.modelMat);
        
        this.topo1.drawit(viewMat, projectionMat, local);
        this.poste1.drawit(viewMat, projectionMat, local);
    } 
    
    
}



class pista extends CGRAobject {
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        //this.numvertices = 2000;
        
        var disco = [];
        var vertices = [];
        var raio = 5; 
        var colors=[];
        this.texCoords = [];
        this.normals = [];
        
        // Criar discos com raios diferentes para fazer a pista
        for (var i = 0; i <= 3; i++){
            disco[i] = [];
            for (var j = 0.0; j <= 2.0*Math.PI; j+=2.0*Math.PI/20){
                disco[i] = disco[i].concat(raio*Math.cos(j), 0, raio*Math.sin(j));
            }
            if(i == 1){
                raio = raio + 20;
            }else{
                raio = raio + 1;
            }

        }
        //console.log(disco)
        //console.log(disco.length)
        
        // Definir cores dos diferentes discos para criar o aspeto de uma estrada
        //var cor = [];
        for (var i=0; i < disco.length-1 ; i++){
            
            // Unir pontos dos discos como realizado para os sólidos
            for (var j = 0; j < disco[0].length; j = j+3){
                vertices = vertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]);
                vertices = vertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);
                vertices = vertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                
                //this.normals = this.normals.concat([disco[i+1][j],disco[i+1][j+1],0]);
                //this.normals = this.normals.concat([disco[i][j],disco[i][j+1],0]);
                //this.normals = this.normals.concat([disco[i][j+3],disco[i][j+4],0]);
                this.normals = this.normals.concat([0,1,0]);
                this.normals = this.normals.concat([0,1,0]);
                this.normals = this.normals.concat([0,1,0]);
                
                if(i==0){ // cinzento
                    colors = colors.concat(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5);
                }else if(i==1){ // preto
                    colors = colors.concat(0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2);
                }else if(i==2){ // cinzento
                    colors = colors.concat(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5);
                }
                
                vertices = vertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]);
                vertices = vertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); 
                vertices = vertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); 
                
                //this.normals = this.normals.concat([disco[i+1][j],disco[i+1][j+1],0]);
                //this.normals = this.normals.concat([disco[i+1][j+3],disco[i+1][j+4],0]);
                //this.normals = this.normals.concat([disco[i][j+3],disco[i][j+4],0]);
                this.normals = this.normals.concat([0,1,0]);
                this.normals = this.normals.concat([0,1,0]);
                this.normals = this.normals.concat([0,1,0]);
                
                if(i==0){ // cinzento
                    colors = colors.concat(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5);
                }else if(i==1){ // preto
                    colors = colors.concat(0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2);
                }else if(i==2){ // cinzento
                    colors = colors.concat(0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5);
                }
            }
        }
        
        
        this.tcoords = [
            glm.vec2(0.0,1.0),
            glm.vec2(0.0,0.0),
            glm.vec2(1.0,0.0)
        ];
        
        for (var j=0; j<vertices.length/3; j=j+3){
           this.texCoords = this.texCoords.concat(this.tcoords[0].array); 
           this.texCoords = this.texCoords.concat(this.tcoords[1].array);
           this.texCoords = this.texCoords.concat(this.tcoords[2].array);
        }
          
        this.numvertices = vertices.length;
                         
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        // TEXTURE COORDS
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(this.texCoords), this.gl.STATIC_DRAW);
        
        // NORMALS
        this.normalsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.normalsbuffer);  
        var a=new Float32Array(this.normals);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
    }
}


class pistaT extends pista{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    setViewPos(viewPos){
        this.viewPos = viewPos;
    }
    
    setLights(lightposit, lightcor){
        this.lightpos = lightposit;
        this.lightcolor = lightcor;
    }
    
    drawit(viewMat, projectionMat,local){
        this.shaderprog.startUsing();
        
        // TEXTURE CORDS
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 2);
        
        // AMBIENT
        var Ambient = glm.vec3(0.5,0.5,0.5);
        this.ambLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram, "Ambient");
        this.gl.uniform3f(this.ambLocation, Ambient.r, Ambient.g, Ambient.b);
        
        // NORMALS
        this.normalsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_normals");
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.normalsbuffer);
        this.gl.vertexAttribPointer(this.normalsLocation, // Attribute location //
                           3, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.normalsLocation);
        
        // NORMAL MATRIX
        var NormalMat = glm.mat3(this.modelMat);
        var matriz_normal = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"NormalMatrix");
        this.gl.uniformMatrix3fv(matriz_normal, false, NormalMat.array);
        
        // LIGHT COLOR -> BRANCA POR PADRÃO
        //var lightColor = glm.vec3(1.0,1.0,1.0);
        var colorLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightColor");
        this.gl.uniform3fv(colorLoc, this.lightcolor.array);
        
        // LIGHT DIRECTION AND POSITION
        //var lightPos = glm.vec3(0.0,100.0,0.0);
        // (PosiçãoDaLuz - PosiçãoDoObjecto), onde o vetor normalizado é a direção e a distância é a norma do vetor
        var lightDir = this.lightpos ['-'] (this.modelMat[3].xyz);
        var lightLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightPosition");
        this.gl.uniform3fv(lightLoc, glm.normalize(this.lightpos).array);
        
        // SHININESS
        var shine = 5.0;
        var shineLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Shininess");
        this.gl.uniform1f(shineLocation, shine);
        
        // STRENGTH
        var strength = 1.0;
        var strengthLocation = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"Strength");
        this.gl.uniform1f(strengthLocation, strength);
        
        // VIEW POSITION (eye da LookAt)
        var viewPosLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ViewPosition");
        this.gl.uniform3fv(viewPosLoc, this.viewPos.array);
        
        // CONSTANT ATTENUATION
        var constAtt = 0.0;
        var constAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"ConstantAttenuation");
        this.gl.uniform1f(constAttLoc, constAtt);
        
        // LINEAR ATTENUATION
        var LinearAtt = 0.25;
        var LinearAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LinearAttenuation");
        this.gl.uniform1f(LinearAttLoc, LinearAtt);
        
        // QUADRATIC ATTENUATION
        var quadAtt = 0.0;
        var quadAttLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"QuadraticAttenuation");
        this.gl.uniform1f(quadAttLoc, constAtt);
        
        // LIGHT DISTANCE -> length(lightDirection)
        //var LightDist = glm.vec3(this.modelMat[3].xyz) ['-'] (direc);
        var lightDLoc = this.gl.getUniformLocation(this.shaderprog.shaderProgram,"LightDistance");
        this.gl.uniform1f(lightDLoc, glm.length(lightDir)); //norma
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat,local);
    } 
}

</script>

## Exercise C.1

Neste exercício era nos pedido para adicionarmos ao trabalho desenvolvido no assignment anterior as modificações necessárias para que fosse possível visualizar a cena em estereoscopia utilizando uma abordagem anaglyph. Para tal, aquilo que fizemos foi definir duas cameras diferentes lado a lado, ou seja, uma para cada olho. Assim sendo, começámos por realizar os cálculos necessários para as cameras que estão presentes na Ficha 12 disponibilizada como material de apoio. Primeiramente calculamos então o vca através do vetor pcam que é um vetor com a localização da camera, e do vetor vup que representa o up vector. Em seguida calculamos o vetor vright que aponta para o "lado direito" do ponto de vista através do produto interno entre os vetores vca e vup. Após isso, normalizámos o vetor vright dando origem a vrightnorm. Tendo o vrightnorm conseguimos calcular a localização de cada uma das cameras através da equação Pleft=pcam-((deyesep/2)* vrightnorm para a camera da esquerda e a mesma equação para a camera da direita alterando apenas a subtração para uma soma. A variável deyesep representa a separação média entre os olhos humanos e foi definido um valor fixo para esta variável com o qual obtivemos o melhor resultado. Com estas localizações, com o vetor pat e com o vetor vup definimos cada uma das matrizes de visualização com glm.lookAt. Tendo as matrizes de vista definidas, apenas tivemos de desenhar o cenário para cada uma das matrizes de vista onde antes de desenharmos cada um dos cenários limpamos o depth buffer e definimos uma máscara para apenas passar a cor vermelha para a vista da esquerda e uma máscara para apenas passar a cor azul (e também a cor verde para o cenário não ficar tão escuro) para o caso da vista da direita. Desta forma o desenho dos dois cenários com as diferentes máscaras vão ficar sobrepostos fazendo com que tenhamos o efeito pretendido. No canvas do cenário final é possível verificar esse resultado.

## Cenário Final

In [ ]:
%%html
<canvas id="myCanvas10" width="600" height="600" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp10 extends DEECapp{
    counter = 0;
    walkingView = true;
    dayTime = true;
    o=true;
    o2=true;
    w=false;
    r=false;
    g=false;
    b=false;
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
        
        var fragsrcL = document.getElementById("my-fragment-shader-light").text;
        var vertsrcL = document.getElementById("my-vertex-shader-light").text;
        
        this.shaderprogL = new DEECshader(this.gl);
        this.shaderprogL.srcShaders(vertsrcL,fragsrcL);
        
        var fragsrcD = document.getElementById("my-fragment-shader-D").text;
        var vertsrcD = document.getElementById("my-vertex-shader-D").text;
        
        this.shaderprogD = new DEECshader(this.gl);
        this.shaderprogD.srcShaders(vertsrcD,fragsrcD);
        
        var fragsrcN = document.getElementById("my-fragment-shader-NoSpecular").text;
        var vertsrcN = document.getElementById("my-vertex-shader-NoSpecular").text;
        
        this.shaderprogN = new DEECshader(this.gl);
        this.shaderprogN.srcShaders(vertsrcN,fragsrcN);
        
        var fragsrcP = document.getElementById("my-fragment-shader-P").text;
        var vertsrcP = document.getElementById("my-vertex-shader-P").text;
        
        this.shaderprogP = new DEECshader(this.gl);
        this.shaderprogP.srcShaders(vertsrcP,fragsrcP);
        
        var fragsrcNP = document.getElementById("my-fragment-shader-NoSpecularP").text;
        var vertsrcNP = document.getElementById("my-vertex-shader-NoSpecularP").text;
        
        this.shaderprogNP = new DEECshader(this.gl);
        this.shaderprogNP.srcShaders(vertsrcNP,fragsrcNP);
  
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        
        
        
        
        // Create a texture to render to
        this.targetTextureWidth = 256;
        this.targetTextureHeight = 256;
        this.targetTexture = this.gl.createTexture();
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.targetTexture);
        
        this.gl.texImage2D(this.gl.TEXTURE_2D, 0, this.gl.RGBA,
                  this.targetTextureWidth, this.targetTextureHeight, 0,
                  this.gl.RGBA, this.gl.UNSIGNED_BYTE, null);

        // set the filtering so we don't need mips
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_MIN_FILTER, this.gl.LINEAR);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_S, this.gl.CLAMP_TO_EDGE);
        this.gl.texParameteri(this.gl.TEXTURE_2D, this.gl.TEXTURE_WRAP_T, this.gl.CLAMP_TO_EDGE);
        

        // Create and bind the framebuffer
        this.fb = this.gl.createFramebuffer();
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);

        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);

        // create a depth renderbuffer
        this.depthBuffer = this.gl.createRenderbuffer();
        this.gl.bindRenderbuffer(this.gl.RENDERBUFFER, this.depthBuffer);

        // make a depth buffer and the same size as the targetTexture
        this.gl.renderbufferStorage(this.gl.RENDERBUFFER, this.gl.DEPTH_COMPONENT16, 
                                  this.targetTextureWidth, this.targetTextureHeight);
        this.gl.framebufferRenderbuffer(this.gl.FRAMEBUFFER, this.gl.DEPTH_ATTACHMENT, 
                                      this.gl.RENDERBUFFER, this.depthBuffer);
        
        
        
         
        
        // ------------------------------- DIA ------------------------------------------
        
        // CARROS
        var rotate90 = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0)));
        this.carro1 = new compoundObjectCar1(this.gl);
        this.carro1.setShader(this.shaderprogD);
        var transformation = glm.translate(glm.vec3(0, 1, -20));
        var transformation2 = transformation['*'](rotate90);
        this.carro1.setModelTransformation(transformation2);
        //this.carro1Pos = glm.vec3(0.0,1.0,-20.0);
        
        this.carro2 = new compoundObjectCar2(this.gl);
        this.carro2.setShader(this.shaderprogD);
        var transformation = glm.translate(glm.vec3(0, 1, -12));
        var transformation2 = transformation['*'](rotate90);
        this.carro2.setModelTransformation(transformation2);
        //this.carro2Pos = glm.vec3(0.0,1.0,-12.0);
             
        // CHÃO  
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("greenish_texture.jpg");
        this.chao = new cubeT(this.gl);
        this.chao.setShader(this.shaderprogN);
        this.chao.settexture(this.texture);
        var scale = glm.scale(glm.vec3(500,0.1,500));
        var transformation = glm.translate(glm.vec3(0,-5,0));
        transformation = transformation['*'](scale);
        this.chao.setModelTransformation(transformation);
        
        // SOL
        this.sol = new esferaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.sol.settexture(this.texture);
        this.sol.setShader(this.shaderprogN);
        var scale = glm.scale(glm.vec3(10,10,10));
        transformation = glm.translate(glm.vec3(100,60,60));
        transformation = transformation['*'](scale);
        this.sol.setModelTransformation(transformation);
        
        // NUVEM 
        this.nuvem1 = new esferaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("nuvem_textura.png");
        this.nuvem1.settexture(this.texture);
        this.nuvem1.setShader(this.shaderprogN);
        var scale = glm.scale(glm.vec3(7,7,7));
        transformation = glm.translate(glm.vec3(0,40,40));
        transformation = transformation['*'](scale);
        this.nuvem1.setModelTransformation(transformation);
        
        this.nuvem2 = new esferaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("nuvem_textura.png");
        this.nuvem2.settexture(this.texture);
        this.nuvem2.setShader(this.shaderprogN);
        var scale = glm.scale(glm.vec3(7,7,7));
        transformation = glm.translate(glm.vec3(10,40,40));
        transformation = transformation['*'](scale);
        this.nuvem2.setModelTransformation(transformation);
        
        this.nuvem3 = new esferaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("nuvem_textura.png");
        this.nuvem3.settexture(this.texture);
        this.nuvem3.setShader(this.shaderprogN);
        var scale = glm.scale(glm.vec3(7,7,7));
        transformation = glm.translate(glm.vec3(20,40,40));
        transformation = transformation['*'](scale);
        this.nuvem3.setModelTransformation(transformation);
        
        // PISTA
        this.pista = new pistaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_preta.jpg");
        this.pista.settexture(this.texture);
        this.pista.setShader(this.shaderprogN);
        
        // META/ARVORES
        this.meta = new compoundObjectArvore(this.gl);
        this.meta.setShader(this.shaderprogN);
        
        
        
        // ------------------------------- NOITE ------------------------------------------
        
        
        // ECRA GRANDE
        this.mycube6 = new cubeT3(this.gl);
        this.mycube6.textureid = this.targetTexture;
        this.mycube6.setShader(this.shaderprogT);
        
        
        
        
        // CARROS
        var rotate90 = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(0.0,1.0,0.0)));
        this.carro1N = new compoundObjectCar1(this.gl);
        this.carro1N.setShader(this.shaderprogP);
        var transformation = glm.translate(glm.vec3(0, 1, -20));
        var transformation2 = transformation['*'](rotate90);
        this.carro1N.setModelTransformation(transformation2);
        //this.carro1Pos = glm.vec3(0.0,1.0,-20.0);
        
        this.carro2N = new compoundObjectCar2(this.gl);
        this.carro2N.setShader(this.shaderprogP);
        var transformation = glm.translate(glm.vec3(0, 1, -12));
        var transformation2 = transformation['*'](rotate90);
        this.carro2N.setModelTransformation(transformation2);
        //this.carro2Pos = glm.vec3(0.0,1.0,-12.0);
        
        // PISTA
        this.pistaN = new pistaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_preta.jpg");
        this.pistaN.settexture(this.texture);
        this.pistaN.setShader(this.shaderprogNP);
             
        // CHÃO  
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("greenish_texture.jpg");
        this.chaoN = new cubeT(this.gl);
        this.chaoN.setShader(this.shaderprogNP);
        this.chaoN.settexture(this.texture);
        var scale = glm.scale(glm.vec3(500,0.1,500));
        var transformation = glm.translate(glm.vec3(0,-5,0));
        transformation = transformation['*'](scale);
        this.chaoN.setModelTransformation(transformation);
        
        // LUA
        this.lua = new esferaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("nuvem_textura.png");
        this.lua.settexture(this.texture);
        this.lua.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(10,10,10));
        transformation = glm.translate(glm.vec3(100,60,60));
        transformation = transformation['*'](scale);
        this.lua.setModelTransformation(transformation);
        
        // ARVORE NATAL
        this.arvorenatal = new compoundObjectArvoreNatal(this.gl);
        this.arvorenatal.setShader(this.shaderprogNP);
        
        // BONECO DE NEVE
        this.boneco = new compoundObjectBoneco(this.gl);
        this.boneco.setShader(this.shaderprogNP);
        
        // ESTRELAS
        this.estrela1 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela1.settexture(this.texture);
        this.estrela1.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(0,40,40));
        transformation = transformation['*'](scale);
        this.estrela1.setModelTransformation(transformation);
        
        this.estrela2 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela2.settexture(this.texture);
        this.estrela2.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(20,50,40));
        transformation = transformation['*'](scale);
        this.estrela2.setModelTransformation(transformation);
        
        this.estrela3 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela3.settexture(this.texture);
        this.estrela3.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(10,30,40));
        transformation = transformation['*'](scale);
        this.estrela3.setModelTransformation(transformation);
        
        this.estrela4 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela4.settexture(this.texture);
        this.estrela4.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(-25,40,40));
        transformation = transformation['*'](scale);
        this.estrela4.setModelTransformation(transformation);
        
        this.estrela5 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela5.settexture(this.texture);
        this.estrela5.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(-20,50,40));
        transformation = transformation['*'](scale);
        this.estrela5.setModelTransformation(transformation);
        
        this.estrela6 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela6.settexture(this.texture);
        this.estrela6.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(-10,30,40));
        transformation = transformation['*'](scale);
        this.estrela6.setModelTransformation(transformation);
        
        this.estrela7 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela7.settexture(this.texture);
        this.estrela7.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(50,40,40));
        transformation = transformation['*'](scale);
        this.estrela7.setModelTransformation(transformation);
        
        this.estrela8 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela8.settexture(this.texture);
        this.estrela8.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(40,50,40));
        transformation = transformation['*'](scale);
        this.estrela8.setModelTransformation(transformation);
        
        this.estrela9 = new estrelaT(this.gl);
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_amarela.jpg");
        this.estrela9.settexture(this.texture);
        this.estrela9.setShader(this.shaderprogNP);
        var scale = glm.scale(glm.vec3(1,1,1));
        transformation = glm.translate(glm.vec3(60,30,40));
        transformation = transformation['*'](scale);
        this.estrela9.setModelTransformation(transformation);
        
        
        
        
        
        // POSTES
        this.poste = new compoundObjectPosteOff(this.gl);
        this.poste.setShader(this.shaderprogN);
        this.poste2 = new compoundObjectPosteOff2(this.gl);
        this.poste2.setShader(this.shaderprogNP);
        this.posteW = new compoundObjectPosteW(this.gl);
        this.posteW.setShader(this.shaderprogNP);
        this.posteR = new compoundObjectPosteR(this.gl);
        this.posteR.setShader(this.shaderprogNP);
        this.posteG = new compoundObjectPosteG(this.gl);
        this.posteG.setShader(this.shaderprogNP);
        this.posteB = new compoundObjectPosteB(this.gl);
        this.posteB.setShader(this.shaderprogNP);
        
        this.posteBranco = new compoundObjectPosteBranco(this.gl);
        this.posteBranco.setShader(this.shaderprogNP);
        
        // POSTES ECRA
        this.texture = new CGRAtexture(this.gl);
        this.texture.load("textura_cinzenta.jpg");
        this.postel = new cilindroT(this.gl);
        this.postel.setShader(this.shaderprogN);
        this.poster = new cilindroT(this.gl);
        this.poster.setShader(this.shaderprogN);
        this.postel.settexture(this.texture);
        this.poster.settexture(this.texture);
        this.postelN = new cilindroT(this.gl);
        this.postelN.setShader(this.shaderprogNP);
        this.posterN = new cilindroT(this.gl);
        this.posterN.setShader(this.shaderprogNP);
        this.postelN.settexture(this.texture);
        this.posterN.settexture(this.texture);
               
        var rotate90 = glm.toMat4(glm.angleAxis(-Math.PI/2.0,glm.vec3(1.0,0.0,0.0)));
        var scale = glm.scale(glm.vec3(0.5,0.7,7.5));
        var transformation = glm.translate(glm.mat4(1.0),glm.vec3(25,0,47.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.postel.setModelTransformation(transformation2);
        
        var transformation = glm.translate(glm.mat4(1.0),glm.vec3(9,0,53));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.poster.setModelTransformation(transformation2);
        
        var transformation = glm.translate(glm.mat4(1.0),glm.vec3(25,0,47.5));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.postelN.setModelTransformation(transformation2);
        
        var transformation = glm.translate(glm.mat4(1.0),glm.vec3(9,0,53));
        var transformation2 = transformation['*'](rotate90)['*'](scale)
        this.posterN.setModelTransformation(transformation2);
        
        
        
        
        

        // Posicoes de projecao e de vista
        this.projectionM = glm.perspective(glm.radians(90),1,0.1,1000);
        
        // Posições iniciais a olhar para o centro da pista
        this.positionx = -28;
        this.positionz = -29;
        this.angle = Math.PI/4;
        
        this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx,4,Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
        
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
        
    }
    
    // Pressionamento de teclas
    keyprocess(evt){
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 97: // a -> Rotaciona para a esquerda -> Alterar a posicao para onde a camera olha (cameraTarget)        
                 this.angle = this.angle + Math.PI/10;    
                 this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx,4,Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
                 console.log('Posicao do visitante rotacionada para a esquerda');
             
                 break;
             case 115: // s -> Rotaciona para a direita -> Alterar a posicao para onde a camera olha (cameraTarget)
                 this.angle = this.angle - Math.PI/10; 
                 this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx,4,Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
                 console.log('Posicao do visitante rotacionada para a direita');
             
                 break;
             case 112: // p -> Move para a frente -> Alterar a posicao da camera
                 this.positionx = this.positionx + Math.sin(this.angle);
                 this.positionz = this.positionz + Math.cos(this.angle);
                 this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx,4,Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
                 console.log('Posicao do visitante movida para a frente');
             
                 break;
             case 108: // l -> Move para trás -> Alterar a posicao da camera
                 this.positionx = this.positionx - Math.sin(this.angle);
                 this.positionz = this.positionz - Math.cos(this.angle);
                 this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx,4,Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
                 console.log('Posicao do visitante movida para tras');
             
                 break;
             case 118: // v -> Alternar entre a vista do condutor e a vista do visitante
                 this.walkingView = !this.walkingView;
                 console.log('Alternado para a vista do condutor');
             
                 break;
             case 110: // n -> Alternar para o cenario noturno
                 this.dayTime = false;
                 console.log('Alternado para o cenario noturno');
             
                 break;
             case 100: // d -> Alternar para o cenario diurno
                 this.dayTime = true;
                 console.log('Alternado para o cenario diurno');
             
                 break;
             case 111: // o -> Alternar para o poste off
                 this.o2=true;
                 this.w=false;
                 this.r=false;
                 this.g=false;
                 this.b=false;
                 console.log('Alternado para o poste off');
             
                 break;
             
             case 119: // w -> Alternar para o poste branco
                 this.o2=false;
                 this.w=true;
                 this.r=false;
                 this.g=false;
                 this.b=false;
                 console.log('Alternado para o poste branco');
             
                 break;
             
             case 114: // r -> Alternar para o poste vermelho
                 this.o2=false;
                 this.w=false;
                 this.r=true;
                 this.g=false;
                 this.b=false;
                 console.log('Alternado para o poste vermelho');
             
                 break;
             
             case 103: // g -> Alternar para o poste verde
                 this.o2=false;
                 this.w=false;
                 this.r=false;
                 this.g=true;
                 this.b=false;
                 console.log('Alternado para o poste verde');
             
                 break;
             
             case 98: // b -> Alternar para o poste azul
                 this.o2=false;
                 this.w=false;
                 this.r=false;
                 this.g=false;
                 this.b=true;
                 console.log('Alternado para o poste azul');
             
                 break;
        }
    }
    
    
    render(){
        this.counter++;

        // Alternar entre a vista do condutor e do visitante
        if (this.walkingView){
            this.viewM = glm.lookAt(glm.vec3(this.positionx,4,this.positionz),glm.vec3(Math.sin(this.angle)+this.positionx, 4, Math.cos(this.angle)+this.positionz),glm.vec3(0,1,0));
            var eyePos = glm.vec3(this.positionx,4,this.positionz);
        }else{
            this.viewM = glm.lookAt(glm.vec3(this.carro1Pos[0], this.carro1Pos[1]+5.5, this.carro1Pos[2]),glm.vec3(this.carro1Pos[0]+Math.cos(-this.counter/20-Math.PI/2), 5.5, this.carro1Pos[2]-Math.sin(-this.counter/20-Math.PI/2)),glm.vec3(0,1,0));
            var eyePos = glm.vec3(this.carro1Pos[0], this.carro1Pos[1]+5.5, this.carro1Pos[2]);
        }
        
        
        
        
        
    
        
        
        // --------------------------------------------------------------------------------
        // Desenhar o cenario para cada uma das versões (azul e vermelha) e sobrepor
        
        this.vup = glm.vec3(0,1,0);
        this.pat = glm.vec3(Math.sin(this.angle)+this.positionx, 4, Math.cos(this.angle)+this.positionz);
        this.pcam = glm.vec3(this.positionx,4,this.positionz);
        
        // pat -> ponto de interesse e pcam -> localizacao do centro de vista
        this.vca = this.pat['-'](this.pcam);
        //console.log("vca = ", this.vca);
        //this.vright = this.vca['*'](this.vup);
        this.vright = glm.cross(this.vca,this.vup);
        //console.log("vright = ", this.vright);
        this.vrightnorm = glm.normalize(this.vright);
        //console.log("vrightnorm = ", this.vrightnorm);
        
        var deyesep = 0.02;
        this.aux = this.vrightnorm['*'](deyesep/2);
        //console.log("aux = ", this.aux);
        this.leftcam = this.pcam['-'](this.aux);
        //console.log("leftcam = ", this.leftcam);
        this.rightcam = this.pcam['+'](this.aux);
        //console.log("rightcam = ", this.rightcam);
        
        this.leftviewM = glm.lookAt(this.leftcam,this.pat,this.vup);
        this.rightviewM = glm.lookAt(this.rightcam,this.pat,this.vup);
        
        this.gl.clearColor(1, 1, 1, 1);   // clear to branco
        //this.gl.clearColor(0, 0, 0, 1);   // clear to preto
        //this.gl.colorMask(this.gl.TRUE,this.gl.TRUE,this.gl.TRUE,this.gl.TRUE);
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
        
        
        // draw scene for left eye (red)
        this.gl.colorMask(1.0,0.0,0.0,1.0);
        
        // set view for the user
        // Tell WebGL how to convert from clip space to pixels

        this.gl.enable(this.gl.DEPTH_TEST);

        // render to our targetTexture by binding the framebuffer
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);
        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);
        
        // Tell WebGL how to convert from clip space to pixels
        this.gl.viewport(0, 0, this.targetTextureWidth, this.targetTextureHeight);


        this.gl.clearColor(0, 1, 0, 1);   // clear to blue
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT); 
                
        // normally the view matrix should be different   
        this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
        this.drawscene(glm.lookAt(glm.vec3(this.carro1Pos[0], this.carro1Pos[1]+5.5, this.carro1Pos[2]),glm.vec3(this.carro1Pos[0]+Math.cos(-this.counter/20-Math.PI/2), 5.5, this.carro1Pos[2]-Math.sin(-this.counter/20-Math.PI/2)),glm.vec3(0,1,0)),this.projectionM,eyePos);
   
        
        // now going to render to the canvas
        
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, null);
        this.gl.enable(this.gl.DEPTH_TEST);

        this.gl.viewport(0, 0, this.gl.canvas.width, this.gl.canvas.height);
        
        
        this.drawscene(this.leftviewM,this.projectionM,eyePos);
        
        
        
        
        // draw scene for right eye (blue and green)
        this.gl.clear(this.gl.DEPTH_BUFFER_BIT);
        this.gl.colorMask(0.0,1.0,1.0,1.0);
        
        // set view for the user
        // Tell WebGL how to convert from clip space to pixels

        this.gl.enable(this.gl.DEPTH_TEST);

        // render to our targetTexture by binding the framebuffer
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, this.fb);
        this.gl.framebufferTexture2D(this.gl.FRAMEBUFFER, this.gl.COLOR_ATTACHMENT0, 
                                   this.gl.TEXTURE_2D, this.targetTexture, 0);
        
        // Tell WebGL how to convert from clip space to pixels
        this.gl.viewport(0, 0, this.targetTextureWidth, this.targetTextureHeight);


        //this.gl.clearColor(0, 1, 0, 1);   // clear to blue
        //this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT); 
                
        // normally the view matrix should be different   
        this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
        this.drawscene(glm.lookAt(glm.vec3(this.carro1Pos[0], this.carro1Pos[1]+5.5, this.carro1Pos[2]),glm.vec3(this.carro1Pos[0]+Math.cos(-this.counter/20-Math.PI/2), 5.5, this.carro1Pos[2]-Math.sin(-this.counter/20-Math.PI/2)),glm.vec3(0,1,0)),this.projectionM,eyePos);
   
        
        // now going to render to the canvas
        
        this.gl.bindFramebuffer(this.gl.FRAMEBUFFER, null);
        this.gl.enable(this.gl.DEPTH_TEST);

        this.gl.viewport(0, 0, this.gl.canvas.width, this.gl.canvas.height);
        
        
        this.drawscene(this.rightviewM,this.projectionM,eyePos);
                       
    }
    
      
    
    
    drawscene(view,proj,eyePos){
        
        if (this.dayTime){ // DIA

            //this.gl.clearColor(0.53,0.77,0.92,1.0);
            this.gl.clearColor(0.00,0.75,0.99,1.0);
            this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        
            // CHÃO
            this.chao.setViewPos(eyePos);
            this.chao.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.chao.drawit(view, proj);

            // SOL
            this.sol.setViewPos(eyePos);
            this.sol.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.sol.drawit(view, proj);

            // NUVEM
            this.nuvem1.setViewPos(eyePos);
            this.nuvem1.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.nuvem1.drawit(view, proj);
            this.nuvem2.setViewPos(eyePos);
            this.nuvem2.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.nuvem2.drawit(view, proj);
            this.nuvem3.setViewPos(eyePos);
            this.nuvem3.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.nuvem3.drawit(view, proj);
         
            // PISTA
            this.pista.setViewPos(eyePos);
            this.pista.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.pista.drawit(view, proj);

            // META/ARVORES
            this.meta.setViewPos(eyePos);
            this.meta.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.meta.drawit(view, proj);
            
            // POSTES
            this.poste.setViewPos(eyePos);
            this.poste.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.poste.drawit(view, proj);
                          

            // CARRO 1 -> Carro vermelho 

            // rotacao para alterar o angulo do carro para alinhar com a pista
            var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

            // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
            this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
            var transformation = glm.translate(this.carro1Pos);
            transformation = (transformation)['*'](rotacao1);
            this.carro1.setModelTransformation(transformation);
            this.carro1.setViewPos(eyePos);
            this.carro1.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.carro1.drawit(view, proj);


            // CARRO 2 -> Carro azul

            // rotacao para alterar o angulo do carro para alinhar com a pista
            var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

            // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
            this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
            var transformation = glm.translate(this.carro2Pos);
            transformation = (transformation)['*'](rotacao2);
            this.carro2.setModelTransformation(transformation);
            this.carro2.setViewPos(eyePos);
            this.carro2.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.carro2.drawit(view, proj);
                          
                          
            this.postel.setViewPos(eyePos); 
            this.postel.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.postel.drawit(view, proj); 
            this.poster.setViewPos(eyePos);
            this.poster.setLights(glm.vec3(0.0,100.0,0.0), glm.vec3(1.0,1.0,1.0)); // posicao da luz e cor branca
            this.poster.drawit(view, proj); 
              
            var scale = glm.scale(glm.vec3(2.5,2.0,0.5));  
            var rotation = glm.rotate(glm.mat4(1.0),glm.radians(17),glm.vec3(0.0,1.0,0.0));
            var transl = glm.translate(glm.mat4(1.0),glm.vec3(17.5,17.5,50.5));
            var transformation = transl['*'](rotation)['*'](scale);
            this.mycube6.setModelTransformation(transformation);
            this.mycube6.drawit(view, proj); 
                          
                          
                          
                          
        }else{ // NOITE

            this.gl.clearColor(0.10,0.10,0.44,1.0);
            this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
            
            
            if(this.o2){  // iluminacao preta/desligada 
                this.chaoN.setViewPos(eyePos);
                this.chaoN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.chaoN.drawit(view, proj);        
                        
                this.lua.setViewPos(eyePos);
                this.lua.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.lua.drawit(view, proj);
                        
                this.pistaN.setViewPos(eyePos);
                this.pistaN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0)); 
                this.pistaN.drawit(view, proj);
                        
                this.arvorenatal.setViewPos(eyePos);
                this.arvorenatal.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.arvorenatal.drawit(view, proj);
                        
                this.boneco.setViewPos(eyePos);
                this.boneco.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.boneco.drawit(view, proj);
                        
                this.postelN.setViewPos(eyePos); 
                this.postelN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.postelN.drawit(view, proj);  
                this.posterN.setViewPos(eyePos);
                this.posterN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0)); 
                this.posterN.drawit(view, proj);
                        
                this.poste2.setViewPos(eyePos);
                this.poste2.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0)); // posicao da luz(poste de iluminacao) e cor preta
                this.poste2.drawit(view, proj);
                this.posteBranco.setViewPos(eyePos);
                this.posteBranco.setLights(glm.vec3(-25,12,18), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteBranco.drawit(view, proj);
                        
                // ESTRELAS
                this.estrela1.setViewPos(eyePos);
                this.estrela1.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela1.drawit(view, proj);
                this.estrela2.setViewPos(eyePos);
                this.estrela2.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela2.drawit(view, proj);
                this.estrela3.setViewPos(eyePos);
                this.estrela3.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela3.drawit(view, proj);
                this.estrela4.setViewPos(eyePos);
                this.estrela4.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela4.drawit(view, proj);
                this.estrela5.setViewPos(eyePos);
                this.estrela5.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela5.drawit(view, proj);
                this.estrela6.setViewPos(eyePos);
                this.estrela6.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela6.drawit(view, proj);
                this.estrela7.setViewPos(eyePos);
                this.estrela7.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela7.drawit(view, proj);
                this.estrela8.setViewPos(eyePos);
                this.estrela8.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela8.drawit(view, proj);
                this.estrela9.setViewPos(eyePos);
                this.estrela9.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.estrela9.drawit(view, proj);
                        
                // CARRO 1 -> Carro vermelho 

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
                this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
                var transformation = glm.translate(this.carro1Pos);
                transformation = (transformation)['*'](rotacao1);
                this.carro1N.setModelTransformation(transformation);
                this.carro1N.setViewPos(eyePos);
                this.carro1N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.carro1N.drawit(view, proj);


                // CARRO 2 -> Carro azul

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
                this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
                var transformation = glm.translate(this.carro2Pos);
                transformation = (transformation)['*'](rotacao2);
                this.carro2N.setModelTransformation(transformation);
                this.carro2N.setViewPos(eyePos);
                this.carro2N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,0.0));
                this.carro2N.drawit(view, proj);
                        
            }
            if(this.w){  // iluminacao branca
                
                this.chaoN.setViewPos(eyePos);
                this.chaoN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.chaoN.drawit(view, proj); 
                       
                this.lua.setViewPos(eyePos);
                this.lua.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.lua.drawit(view, proj);       
                       
                this.pistaN.setViewPos(eyePos);
                this.pistaN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0)); 
                this.pistaN.drawit(view, proj);
                       
                this.arvorenatal.setViewPos(eyePos);
                this.arvorenatal.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.arvorenatal.drawit(view, proj);
                       
                this.boneco.setViewPos(eyePos);
                this.boneco.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.boneco.drawit(view, proj);
                       
                this.postelN.setViewPos(eyePos); 
                this.postelN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.postelN.drawit(view, proj);  
                this.posterN.setViewPos(eyePos);
                this.posterN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0)); 
                this.posterN.drawit(view, proj);
                       
                       
                this.posteW.setViewPos(eyePos);
                this.posteW.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteW.drawit(view, proj);
                this.posteBranco.setViewPos(eyePos);
                this.posteBranco.setLights(glm.vec3(-25,12,18), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteBranco.drawit(view, proj);
                       
                // ESTRELAS
                this.estrela1.setViewPos(eyePos);
                this.estrela1.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela1.drawit(view, proj);
                this.estrela2.setViewPos(eyePos);
                this.estrela2.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela2.drawit(view, proj);
                this.estrela3.setViewPos(eyePos);
                this.estrela3.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela3.drawit(view, proj);
                this.estrela4.setViewPos(eyePos);
                this.estrela4.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela4.drawit(view, proj);
                this.estrela5.setViewPos(eyePos);
                this.estrela5.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela5.drawit(view, proj);
                this.estrela6.setViewPos(eyePos);
                this.estrela6.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela6.drawit(view, proj);
                this.estrela7.setViewPos(eyePos);
                this.estrela7.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela7.drawit(view, proj);
                this.estrela8.setViewPos(eyePos);
                this.estrela8.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela8.drawit(view, proj);
                this.estrela9.setViewPos(eyePos);
                this.estrela9.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.estrela9.drawit(view, proj);      
                       
                // CARRO 1 -> Carro vermelho 

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
                this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
                var transformation = glm.translate(this.carro1Pos);
                transformation = (transformation)['*'](rotacao1);
                this.carro1N.setModelTransformation(transformation);
                this.carro1N.setViewPos(eyePos);
                this.carro1N.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.carro1N.drawit(view, proj);


                // CARRO 2 -> Carro azul

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
                this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
                var transformation = glm.translate(this.carro2Pos);
                transformation = (transformation)['*'](rotacao2);
                this.carro2N.setModelTransformation(transformation);
                this.carro2N.setViewPos(eyePos);
                this.carro2N.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,1.0,1.0));
                this.carro2N.drawit(view, proj);       
                       
            }
            if(this.r){  // iluminacao vermelha
                 
                this.chaoN.setViewPos(eyePos);
                this.chaoN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.chaoN.drawit(view, proj);        
                       
                this.lua.setViewPos(eyePos);
                this.lua.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.lua.drawit(view, proj);       
                       
                this.pistaN.setViewPos(eyePos);
                this.pistaN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0)); 
                this.pistaN.drawit(view, proj);
                       
                this.arvorenatal.setViewPos(eyePos);
                this.arvorenatal.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.arvorenatal.drawit(view, proj);
                       
                this.boneco.setViewPos(eyePos);
                this.boneco.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.boneco.drawit(view, proj);
                       
                       
                this.postelN.setViewPos(eyePos); 
                this.postelN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.postelN.drawit(view, proj);  
                this.posterN.setViewPos(eyePos);
                this.posterN.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0)); 
                this.posterN.drawit(view, proj);
                       
                this.posteR.setViewPos(eyePos);
                this.posteR.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0)); // posicao da luz(poste de iluminacao) e cor vermelha
                this.posteR.drawit(view, proj);
                this.posteBranco.setViewPos(eyePos);
                this.posteBranco.setLights(glm.vec3(-25,12,18), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteBranco.drawit(view, proj);
                       
                // ESTRELAS
                this.estrela1.setViewPos(eyePos);
                this.estrela1.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela1.drawit(view, proj);
                this.estrela2.setViewPos(eyePos);
                this.estrela2.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela2.drawit(view, proj);
                this.estrela3.setViewPos(eyePos);
                this.estrela3.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela3.drawit(view, proj);
                this.estrela4.setViewPos(eyePos);
                this.estrela4.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela4.drawit(view, proj);
                this.estrela5.setViewPos(eyePos);
                this.estrela5.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela5.drawit(view, proj);
                this.estrela6.setViewPos(eyePos);
                this.estrela6.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela6.drawit(view, proj);
                this.estrela7.setViewPos(eyePos);
                this.estrela7.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela7.drawit(view, proj);
                this.estrela8.setViewPos(eyePos);
                this.estrela8.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela8.drawit(view, proj);
                this.estrela9.setViewPos(eyePos);
                this.estrela9.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.estrela9.drawit(view, proj);       
                       
                // CARRO 1 -> Carro vermelho 

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
                this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
                var transformation = glm.translate(this.carro1Pos);
                transformation = (transformation)['*'](rotacao1);
                this.carro1N.setModelTransformation(transformation);
                this.carro1N.setViewPos(eyePos);
                this.carro1N.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.carro1N.drawit(view, proj);


                // CARRO 2 -> Carro azul

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
                this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
                var transformation = glm.translate(this.carro2Pos);
                transformation = (transformation)['*'](rotacao2);
                this.carro2N.setModelTransformation(transformation);
                this.carro2N.setViewPos(eyePos);
                this.carro2N.setLights(glm.vec3(27,12,-1), glm.vec3(1.0,0.0,0.0));
                this.carro2N.drawit(view, proj);
                       
            }
            if(this.g){  // iluminacao verde
                       
                this.chaoN.setViewPos(eyePos);
                this.chaoN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.chaoN.drawit(view, proj);        
                       
                this.lua.setViewPos(eyePos);
                this.lua.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.lua.drawit(view, proj);       
                       
                this.pistaN.setViewPos(eyePos);
                this.pistaN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0)); 
                this.pistaN.drawit(view, proj);
                       
                this.arvorenatal.setViewPos(eyePos);
                this.arvorenatal.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.arvorenatal.drawit(view, proj);
                       
                this.boneco.setViewPos(eyePos);
                this.boneco.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.boneco.drawit(view, proj);
                       
                this.postelN.setViewPos(eyePos); 
                this.postelN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.postelN.drawit(view, proj);  
                this.posterN.setViewPos(eyePos);
                this.posterN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0)); 
                this.posterN.drawit(view, proj);
         
                this.posteG.setViewPos(eyePos);
                this.posteG.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0)); // posicao da luz(poste de iluminacao) e cor verde
                this.posteG.drawit(view, proj);
                this.posteBranco.setViewPos(eyePos);
                this.posteBranco.setLights(glm.vec3(-25,12,18), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteBranco.drawit(view, proj);
                       
                // ESTRELAS
                this.estrela1.setViewPos(eyePos);
                this.estrela1.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela1.drawit(view, proj);
                this.estrela2.setViewPos(eyePos);
                this.estrela2.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela2.drawit(view, proj);
                this.estrela3.setViewPos(eyePos);
                this.estrela3.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela3.drawit(view, proj);
                this.estrela4.setViewPos(eyePos);
                this.estrela4.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela4.drawit(view, proj);
                this.estrela5.setViewPos(eyePos);
                this.estrela5.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela5.drawit(view, proj);
                this.estrela6.setViewPos(eyePos);
                this.estrela6.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela6.drawit(view, proj);
                this.estrela7.setViewPos(eyePos);
                this.estrela7.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela7.drawit(view, proj);
                this.estrela8.setViewPos(eyePos);
                this.estrela8.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela8.drawit(view, proj);
                this.estrela9.setViewPos(eyePos);
                this.estrela9.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.estrela9.drawit(view, proj);
                       
                // CARRO 1 -> Carro vermelho 

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
                this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
                var transformation = glm.translate(this.carro1Pos);
                transformation = (transformation)['*'](rotacao1);
                this.carro1N.setModelTransformation(transformation);
                this.carro1N.setViewPos(eyePos);
                this.carro1N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.carro1N.drawit(view, proj);


                // CARRO 2 -> Carro azul

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
                this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
                var transformation = glm.translate(this.carro2Pos);
                transformation = (transformation)['*'](rotacao2);
                this.carro2N.setModelTransformation(transformation);
                this.carro2N.setViewPos(eyePos);
                this.carro2N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,1.0,0.0));
                this.carro2N.drawit(view, proj);
                       
            }
            if(this.b){  // iluminacao azul
               
                this.chaoN.setViewPos(eyePos);
                this.chaoN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.chaoN.drawit(view, proj); 
                       
                this.lua.setViewPos(eyePos);
                this.lua.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.lua.drawit(view, proj);       
                                 
                this.pistaN.setViewPos(eyePos);
                this.pistaN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0)); 
                this.pistaN.drawit(view, proj); 
                       
                this.arvorenatal.setViewPos(eyePos);
                this.arvorenatal.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.arvorenatal.drawit(view, proj);
                       
                this.boneco.setViewPos(eyePos);
                this.boneco.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.boneco.drawit(view, proj);
                       
                this.postelN.setViewPos(eyePos); 
                this.postelN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.postelN.drawit(view, proj);  
                this.posterN.setViewPos(eyePos);
                this.posterN.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0)); 
                this.posterN.drawit(view, proj);      
                       
                this.posteB.setViewPos(eyePos);
                this.posteB.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0)); // posicao da luz(poste de iluminacao) e cor azul
                //this.posteB.setLights(glm.vec3(-25,12,18), glm.vec3(0.0,0.0,1.0)); // posicao da luz(poste de iluminacao) e cor azul
                this.posteB.drawit(view, proj);
                this.posteBranco.setViewPos(eyePos);
                this.posteBranco.setLights(glm.vec3(-25,12,18), glm.vec3(1.0,1.0,1.0)); // posicao da luz(poste de iluminacao) e cor branca
                this.posteBranco.drawit(view, proj);
                       
                // ESTRELAS
                this.estrela1.setViewPos(eyePos);
                this.estrela1.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela1.drawit(view, proj);
                this.estrela2.setViewPos(eyePos);
                this.estrela2.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela2.drawit(view, proj);
                this.estrela3.setViewPos(eyePos);
                this.estrela3.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela3.drawit(view, proj);
                this.estrela4.setViewPos(eyePos);
                this.estrela4.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela4.drawit(view, proj);
                this.estrela5.setViewPos(eyePos);
                this.estrela5.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela5.drawit(view, proj);
                this.estrela6.setViewPos(eyePos);
                this.estrela6.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela6.drawit(view, proj);
                this.estrela7.setViewPos(eyePos);
                this.estrela7.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela7.drawit(view, proj);
                this.estrela8.setViewPos(eyePos);
                this.estrela8.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela8.drawit(view, proj);
                this.estrela9.setViewPos(eyePos);
                this.estrela9.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.estrela9.drawit(view, proj);
                       
                // CARRO 1 -> Carro vermelho 

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao1 = glm.toMat4(glm.angleAxis(-this.counter/20, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 1 ao percorrer a pista (vetor de translação)
                this.carro1Pos = glm.vec3(20*Math.cos(this.counter/20), 1, 20*Math.sin(this.counter/20));
                var transformation = glm.translate(this.carro1Pos);
                transformation = (transformation)['*'](rotacao1);
                this.carro1N.setModelTransformation(transformation);
                this.carro1N.setViewPos(eyePos);
                this.carro1N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.carro1N.drawit(view, proj);


                // CARRO 2 -> Carro azul

                // rotacao para alterar o angulo do carro para alinhar com a pista
                var rotacao2 = glm.toMat4(glm.angleAxis(-this.counter/10, glm.vec3(0.0,1.0,0.0)));

                // variação da posição do carro 2 ao percorrer a pista (vetor de translação com atraso de 5)
                this.carro2Pos = glm.vec3(10*Math.cos((this.counter-5)/10), 1, 10*Math.sin((this.counter-5)/10));
                var transformation = glm.translate(this.carro2Pos);
                transformation = (transformation)['*'](rotacao2);
                this.carro2N.setModelTransformation(transformation);
                this.carro2N.setViewPos(eyePos);
                this.carro2N.setLights(glm.vec3(27,12,-1), glm.vec3(0.0,0.0,1.0));
                this.carro2N.drawit(view, proj);       
                
            }

              
            var scale = glm.scale(glm.vec3(2.5,2.0,0.5));  
            var rotation = glm.rotate(glm.mat4(1.0),glm.radians(17),glm.vec3(0.0,1.0,0.0));
            var transl = glm.translate(glm.mat4(1.0),glm.vec3(17.5,17.5,50.5));
            var transformation = transl['*'](rotation)['*'](scale);
            this.mycube6.setModelTransformation(transformation);
            this.mycube6.drawit(view, proj);  
        }
            
    }  
    
}


var app10 = new myapp10('myCanvas10');

app10.run();

</script>